In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import joblib
import warnings
warnings.filterwarnings('ignore')

class BlueBikesRFPipeline:
    """
    End-to-end pipeline for Bluebikes demand prediction using Random Forest.
    Optimized for large datasets (6M+ rows) with proper time-based validation.
    """
    
    def __init__(self, pickle_path, sample_size=None, random_state=42, 
                 needs_preprocessing=True):
        """
        Initialize pipeline
        
        Parameters:
        -----------
        pickle_path : str
            Path to data pickle file (raw or preprocessed)
        sample_size : int or float, optional
            If int: number of rows to sample
            If float: fraction of data to sample (e.g., 0.1 for 10%)
            If None: use all data
        random_state : int
            Random seed for reproducibility
        needs_preprocessing : bool
            If True, expects raw trip data and will aggregate/engineer features
            If False, expects already preprocessed data
        """
        self.pickle_path = pickle_path
        self.sample_size = sample_size
        self.random_state = random_state
        self.needs_preprocessing = needs_preprocessing
        self.pickup_model = None
        self.dropoff_model = None
        self.feature_names = None
        self.results = {}
        self.raw_df = None
        self.df = None
        
    def load_data(self):
        """Load and optionally sample data"""
        print("Loading data...")
        with open(self.pickle_path, 'rb') as f:
            self.raw_df = pickle.load(f)
        
        print(f"Original data shape: {self.raw_df.shape}")
        print(f"Columns: {list(self.raw_df.columns)}")
        
        # Sample data if specified (sample BEFORE preprocessing for speed)
        if self.sample_size is not None:
            if isinstance(self.sample_size, float):
                n_samples = int(len(self.raw_df) * self.sample_size)
            else:
                n_samples = self.sample_size
            
            print(f"Sampling {n_samples:,} rows for faster training...")
            self.raw_df = self.raw_df.sample(n=n_samples, random_state=self.random_state)
            print(f"Sampled data shape: {self.raw_df.shape}")
        
        # Preprocess if needed
        if self.needs_preprocessing:
            self.df = self.preprocess_data()
        else:
            self.df = self.raw_df
        
        return self.df
    
    def preprocess_data(self):
        """
        Transform raw trip data into aggregated station-level demand data
        with engineered features
        """
        print("\n" + "="*60)
        print("PREPROCESSING RAW TRIP DATA")
        print("="*60)
        
        df = self.raw_df.copy()
        
        # Ensure datetime columns
        if not pd.api.types.is_datetime64_any_dtype(df['start_time']):
            print("Converting start_time to datetime...")
            df['start_time'] = pd.to_datetime(df['start_time'])
        if not pd.api.types.is_datetime64_any_dtype(df['stop_time']):
            print("Converting stop_time to datetime...")
            df['stop_time'] = pd.to_datetime(df['stop_time'])
        
        # Step 1: Aggregate to hourly station-level pickups
        print("\nAggregating pickups by station and hour...")
        pickups = df.groupby([
            pd.Grouper(key='start_time', freq='1H'),
            'start_station_id'
        ]).agg({
            'ride_id': 'count',
            'start_station_latitude': 'first',
            'start_station_longitude': 'first',
            'user_type': lambda x: (x == 'member').sum() / len(x) if len(x) > 0 else 0,
        }).reset_index()
        
        pickups.columns = ['time', 'station_id', 'pickups', 'latitude', 
                          'longitude', 'member_ratio']
        
        # Step 2: Aggregate dropoffs by station and hour
        print("Aggregating dropoffs by station and hour...")
        dropoffs = df.groupby([
            pd.Grouper(key='stop_time', freq='1H'),
            'end_station_id'
        ]).agg({
            'ride_id': 'count'
        }).reset_index()
        
        dropoffs.columns = ['time', 'station_id', 'dropoffs']
        
        # Step 3: Merge pickups and dropoffs
        print("Merging pickups and dropoffs...")
        station_demand = pd.merge(
            pickups, 
            dropoffs, 
            on=['time', 'station_id'], 
            how='outer'
        ).fillna(0)
        
        # Calculate net flow
        station_demand['net_flow'] = (station_demand['dropoffs'] - 
                                      station_demand['pickups'])
        
        # Step 4: Engineer temporal features
        print("Engineering temporal features...")
        station_demand['hour'] = station_demand['time'].dt.hour
        station_demand['day_of_week'] = station_demand['time'].dt.dayofweek
        station_demand['month'] = station_demand['time'].dt.month
        station_demand['day_of_year'] = station_demand['time'].dt.dayofyear
        station_demand['week_of_year'] = station_demand['time'].dt.isocalendar().week
        
        # Binary features
        station_demand['is_weekend'] = (station_demand['day_of_week'] >= 5).astype(int)
        station_demand['is_rush_hour_morning'] = station_demand['hour'].isin([7, 8, 9]).astype(int)
        station_demand['is_rush_hour_evening'] = station_demand['hour'].isin([17, 18, 19]).astype(int)
        station_demand['is_business_hours'] = station_demand['hour'].between(9, 17).astype(int)
        station_demand['is_night'] = ((station_demand['hour'] >= 22) | 
                                      (station_demand['hour'] <= 5)).astype(int)
        
        # Cyclical encoding for hour and day_of_week
        station_demand['hour_sin'] = np.sin(2 * np.pi * station_demand['hour'] / 24)
        station_demand['hour_cos'] = np.cos(2 * np.pi * station_demand['hour'] / 24)
        station_demand['dow_sin'] = np.sin(2 * np.pi * station_demand['day_of_week'] / 7)
        station_demand['dow_cos'] = np.cos(2 * np.pi * station_demand['day_of_week'] / 7)
        
        # Step 5: Engineer lag features (CRITICAL for time series)
        print("Engineering lag features (this may take a moment)...")
        station_demand = station_demand.sort_values(['station_id', 'time'])
        
        # Create lag features for each station
        for lag in [1, 24, 168]:  # 1 hour, 1 day, 1 week
            station_demand[f'pickups_lag_{lag}h'] = station_demand.groupby('station_id')['pickups'].shift(lag)
            station_demand[f'dropoffs_lag_{lag}h'] = station_demand.groupby('station_id')['dropoffs'].shift(lag)
        
        # Rolling averages
        for window in [24, 168]:  # 1 day, 1 week
            station_demand[f'pickups_rolling_{window}h'] = (
                station_demand.groupby('station_id')['pickups']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
            station_demand[f'dropoffs_rolling_{window}h'] = (
                station_demand.groupby('station_id')['dropoffs']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
        
        # Step 6: Station-level features
        print("Engineering station-level features...")
        station_stats = station_demand.groupby('station_id').agg({
            'pickups': ['mean', 'std', 'max'],
            'dropoffs': ['mean', 'std', 'max'],
        })
        station_stats.columns = ['_'.join(col) for col in station_stats.columns]
        station_stats = station_stats.reset_index()
        
        station_demand = station_demand.merge(station_stats, on='station_id', how='left')
        
        # Step 7: Filter out very low activity periods
        print("Filtering low-activity periods...")
        initial_rows_filter = len(station_demand)
        
        # Keep only stations with at least some minimum activity
        station_total_activity = station_demand.groupby('station_id')[['pickups', 'dropoffs']].sum()
        active_stations = station_total_activity[
            (station_total_activity['pickups'] > 10) | 
            (station_total_activity['dropoffs'] > 10)
        ].index
        
        station_demand = station_demand[station_demand['station_id'].isin(active_stations)]
        filtered_rows = initial_rows_filter - len(station_demand)
        
        print(f"  Filtered out {filtered_rows:,} rows from inactive stations")
        
        # Drop rows with NaN (from lag features at the beginning)
        initial_rows = len(station_demand)
        station_demand = station_demand.dropna()
        dropped_rows = initial_rows - len(station_demand)
        
        print(f"\nPreprocessing complete!")
        print(f"  Original trips: {len(df):,}")
        print(f"  Aggregated to: {len(station_demand):,} station-hour records")
        print(f"  Active stations: {len(active_stations)}")
        print(f"  Dropped {dropped_rows:,} rows due to lag features")
        print(f"  Final features: {len(station_demand.columns)} columns")
        print(f"  Date range: {station_demand['time'].min()} to {station_demand['time'].max()}")
        
        return station_demand
    
    def prepare_features(self, target_col='pickups'):
        """
        Prepare features and target variable
        
        Parameters:
        -----------
        target_col : str
            Target variable name ('pickups', 'dropoffs', or 'net_flow')
        """
        print(f"\nPreparing features for {target_col} prediction...")
        
        # Check if target exists
        if target_col not in self.df.columns:
            raise ValueError(f"Target column '{target_col}' not found in data. "
                           f"Available columns: {list(self.df.columns)}")
        
        # Identify feature columns (exclude targets, identifiers, and time)
        exclude_cols = ['pickups', 'dropoffs', 'net_flow', 'ride_id', 
                       'start_time', 'stop_time', 'start_station_name', 
                       'end_station_name', 'time', 'station_id']
        
        self.feature_names = [col for col in self.df.columns 
                             if col not in exclude_cols and 
                             not self.df[col].isna().all()]
        
        X = self.df[self.feature_names]
        y = self.df[target_col]
        
        print(f"Features: {len(self.feature_names)}")
        print(f"Target: {target_col}")
        print(f"Target range: {y.min():.1f} to {y.max():.1f}")
        print(f"Target mean: {y.mean():.2f}")
        print(f"\nFeature categories:")
        print(f"  - Temporal: hour, day_of_week, month, etc.")
        print(f"  - Lag features: pickups_lag_*, dropoffs_lag_*")
        print(f"  - Rolling features: *_rolling_*")
        print(f"  - Station stats: *_mean, *_std, *_max")
        print(f"  - Location: latitude, longitude")
        
        return X, y
    
    def split_data(self, X, y, test_size=0.2, time_based=True):
        """
        Split data into train/test sets
        
        Parameters:
        -----------
        test_size : float
            Proportion of data for testing
        time_based : bool
            If True, use time-based split (HIGHLY recommended for time series)
            If False, use random split (NOT recommended)
        """
        print(f"\nSplitting data (test_size={test_size})...")
        
        if time_based and 'time' in self.df.columns:
            # Sort by time first
            sorted_df = self.df.sort_values('time').reset_index(drop=True)
            
            # Calculate split point
            split_idx = int(len(sorted_df) * (1 - test_size))
            
            # Get indices for train/test
            train_indices = sorted_df.index[:split_idx]
            test_indices = sorted_df.index[split_idx:]
            
            # Split using original indices
            X_train = X.iloc[train_indices]
            X_test = X.iloc[test_indices]
            y_train = y.iloc[train_indices]
            y_test = y.iloc[test_indices]
            
            # Get time ranges for reporting
            train_times = sorted_df.loc[train_indices, 'time']
            test_times = sorted_df.loc[test_indices, 'time']
            
            print(f"✓ Using time-based split (RECOMMENDED)")
            print(f"  Train period: {train_times.min()} to {train_times.max()}")
            print(f"  Test period:  {test_times.min()} to {test_times.max()}")
        else:
            # Random split
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=self.random_state
            )
            print("⚠️  Using random split (NOT recommended for time series)")
        
        print(f"Train set: {X_train.shape[0]:,} rows")
        print(f"Test set: {X_test.shape[0]:,} rows")
        
        return X_train, X_test, y_train, y_test
    
    def train_model(self, X_train, y_train, target_name='pickups', 
                   n_estimators=100, max_depth=20, min_samples_split=20,
                   min_samples_leaf=10, max_features='sqrt', n_jobs=-1):
        """
        Train Random Forest model with optimized parameters for large datasets
        """
        print(f"\n{'='*60}")
        print(f"Training Random Forest for {target_name}")
        print(f"{'='*60}")
        
        start_time = datetime.now()
        
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            n_jobs=n_jobs,
            random_state=self.random_state,
            verbose=1,
            warm_start=False,
            oob_score=True
        )
        
        print(f"\nModel parameters:")
        print(f"  - Trees: {n_estimators}")
        print(f"  - Max depth: {max_depth}")
        print(f"  - Min samples split: {min_samples_split}")
        print(f"  - Min samples leaf: {min_samples_leaf}")
        print(f"  - Max features: {max_features}")
        print(f"  - Parallel jobs: {n_jobs}")
        
        model.fit(X_train, y_train)
        
        training_time = (datetime.now() - start_time).total_seconds()
        print(f"\n✓ Training completed in {training_time:.2f} seconds")
        print(f"  OOB Score (R²): {model.oob_score_:.4f}")
        
        return model, training_time
    
    def evaluate_model(self, model, X_test, y_test, target_name='pickups'):
        """Evaluate model performance"""
        print(f"\nEvaluating {target_name} model...")
        
        # Predictions
        start_time = datetime.now()
        y_pred = model.predict(X_test)
        prediction_time = (datetime.now() - start_time).total_seconds()
        
        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1))) * 100
        
        results = {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'prediction_time': prediction_time,
            'predictions': y_pred,
            'actuals': y_test
        }
        
        print(f"\n{target_name.upper()} Model Performance:")
        print(f"{'='*50}")
        print(f"  MAE:  {mae:.2f}")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  R²:   {r2:.4f}")
        print(f"  MAPE: {mape:.2f}%")
        print(f"  Prediction time: {prediction_time:.2f}s for {len(X_test):,} samples")
        
        return results
    
    def plot_feature_importance(self, model, target_name='pickups', top_n=20):
        """Plot top N most important features"""
        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False).head(top_n)
        
        plt.figure(figsize=(10, 6))
        sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
        plt.title(f'Top {top_n} Feature Importances - {target_name}')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.savefig(f'feature_importance_{target_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"\n✓ Feature importance plot saved as 'feature_importance_{target_name}.png'")
        
        return importance_df
    
    def plot_predictions(self, results, target_name='pickups', n_samples=1000):
        """Plot actual vs predicted values"""
        if len(results['actuals']) > n_samples:
            indices = np.random.choice(len(results['actuals']), n_samples, replace=False)
            actuals_plot = results['actuals'].iloc[indices]
            predictions_plot = results['predictions'][indices]
        else:
            actuals_plot = results['actuals']
            predictions_plot = results['predictions']
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Scatter plot
        axes[0].scatter(actuals_plot, predictions_plot, alpha=0.5, s=10)
        axes[0].plot([actuals_plot.min(), actuals_plot.max()], 
                     [actuals_plot.min(), actuals_plot.max()], 
                     'r--', lw=2, label='Perfect prediction')
        axes[0].set_xlabel('Actual')
        axes[0].set_ylabel('Predicted')
        axes[0].set_title(f'Actual vs Predicted - {target_name}')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Residuals
        residuals = actuals_plot - predictions_plot
        axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
        axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
        axes[1].set_xlabel('Residual (Actual - Predicted)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title(f'Residuals Distribution - {target_name}')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'predictions_{target_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Prediction plots saved as 'predictions_{target_name}.png'")
    
    def save_model(self, model, target_name='pickups'):
        """Save trained model"""
        filename = f'rf_model_{target_name}.joblib'
        joblib.dump(model, filename)
        print(f"\n✓ Model saved as '{filename}'")
    
    def run_full_pipeline(self, train_pickups=True, train_dropoffs=True):
        """
        Run complete pipeline for both pickups and dropoffs prediction
        """
        print("\n" + "="*60)
        print("BLUEBIKES DEMAND PREDICTION PIPELINE")
        print("="*60)
        
        # Load data
        self.load_data()
        
        # Train pickups model
        if train_pickups and 'pickups' in self.df.columns:
            X, y = self.prepare_features(target_col='pickups')
            X_train, X_test, y_train, y_test = self.split_data(X, y, time_based=True)
            
            self.pickup_model, train_time = self.train_model(
                X_train, y_train, target_name='pickups'
            )
            
            self.results['pickups'] = self.evaluate_model(
                self.pickup_model, X_test, y_test, target_name='pickups'
            )
            
            self.plot_feature_importance(self.pickup_model, target_name='pickups')
            self.plot_predictions(self.results['pickups'], target_name='pickups')
            self.save_model(self.pickup_model, target_name='pickups')
        
        # Train dropoffs model
        if train_dropoffs and 'dropoffs' in self.df.columns:
            X, y = self.prepare_features(target_col='dropoffs')
            X_train, X_test, y_train, y_test = self.split_data(X, y, time_based=True)
            
            self.dropoff_model, train_time = self.train_model(
                X_train, y_train, target_name='dropoffs'
            )
            
            self.results['dropoffs'] = self.evaluate_model(
                self.dropoff_model, X_test, y_test, target_name='dropoffs'
            )
            
            self.plot_feature_importance(self.dropoff_model, target_name='dropoffs')
            self.plot_predictions(self.results['dropoffs'], target_name='dropoffs')
            self.save_model(self.dropoff_model, target_name='dropoffs')
        
        # Calculate net flow if both models trained
        if train_pickups and train_dropoffs and self.pickup_model and self.dropoff_model:
            print("\n" + "="*60)
            print("NET FLOW ANALYSIS (Dropoffs - Pickups)")
            print("="*60)
            
            net_flow_pred = (self.results['dropoffs']['predictions'] - 
                           self.results['pickups']['predictions'])
            net_flow_actual = (self.results['dropoffs']['actuals'].values - 
                             self.results['pickups']['actuals'].values)
            
            mae_netflow = mean_absolute_error(net_flow_actual, net_flow_pred)
            rmse_netflow = np.sqrt(mean_squared_error(net_flow_actual, net_flow_pred))
            r2_netflow = r2_score(net_flow_actual, net_flow_pred)
            
            print(f"\nNet Flow Prediction Performance:")
            print(f"  MAE:  {mae_netflow:.2f}")
            print(f"  RMSE: {rmse_netflow:.2f}")
            print(f"  R²:   {r2_netflow:.4f}")
        
        print("\n" + "="*60)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*60)
        
        return self.results

In [1]:
# Date range input for training
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import joblib
import warnings
warnings.filterwarnings('ignore')

class BlueBikesRFPipeline:
    """
    End-to-end pipeline for Bluebikes demand prediction using Random Forest.
    Optimized for large datasets (6M+ rows) with proper time-based validation.
    """
    
    def __init__(self, pickle_path, sample_size=None, random_state=42, 
                 needs_preprocessing=True, train_start_date=None, train_end_date=None):
        """
        Initialize pipeline
        
        Parameters:
        -----------
        pickle_path : str
            Path to data pickle file (raw or preprocessed)
        sample_size : int or float, optional
            If int: number of rows to sample
            If float: fraction of data to sample (e.g., 0.1 for 10%)
            If None: use all data
        random_state : int
            Random seed for reproducibility
        needs_preprocessing : bool
            If True, expects raw trip data and will aggregate/engineer features
            If False, expects already preprocessed data
        train_start_date : str, optional
            Start date for training data (e.g., '2025-01-01')
            If None, uses earliest date in data
        train_end_date : str, optional
            End date for training data (e.g., '2025-06-30')
            If None, uses latest date in data
        """
        self.pickle_path = pickle_path
        self.sample_size = sample_size
        self.random_state = random_state
        self.needs_preprocessing = needs_preprocessing
        self.train_start_date = pd.to_datetime(train_start_date, utc=True) if train_start_date else None
        self.train_end_date = pd.to_datetime(train_end_date, utc=True) if train_end_date else None
        self.pickup_model = None
        self.dropoff_model = None
        self.feature_names = None
        self.results = {}
        self.raw_df = None
        self.df = None
        
        print(f"\n{'='*70}")
        print(f"BLUEBIKES RANDOM FOREST PIPELINE")
        print(f"{'='*70}")
        print(f"Data: {pickle_path}")
        print(f"Sample size: {sample_size if sample_size else 'ALL (no sampling)'}")
        if train_start_date:
            print(f"Training start: {train_start_date}")
        else:
            print(f"Training start: Earliest date in data")
        if train_end_date:
            print(f"Training end: {train_end_date}")
        else:
            print(f"Training end: Latest date in data")
        print(f"{'='*70}\n")
        
    def load_data(self):
        """Load and optionally sample data"""
        print("Loading data...")
        with open(self.pickle_path, 'rb') as f:
            self.raw_df = pickle.load(f)
        
        print(f"Original data shape: {self.raw_df.shape}")
        print(f"Columns: {list(self.raw_df.columns)}")
        
        # Ensure datetime columns are timezone-aware
        if not pd.api.types.is_datetime64_any_dtype(self.raw_df['start_time']):
            self.raw_df['start_time'] = pd.to_datetime(self.raw_df['start_time'], utc=True)
        elif self.raw_df['start_time'].dt.tz is None:
            self.raw_df['start_time'] = self.raw_df['start_time'].dt.tz_localize('UTC')
            
        if not pd.api.types.is_datetime64_any_dtype(self.raw_df['stop_time']):
            self.raw_df['stop_time'] = pd.to_datetime(self.raw_df['stop_time'], utc=True)
        elif self.raw_df['stop_time'].dt.tz is None:
            self.raw_df['stop_time'] = self.raw_df['stop_time'].dt.tz_localize('UTC')
        
        print(f"Full date range: {self.raw_df['start_time'].min()} to {self.raw_df['start_time'].max()}")
        
        # Filter by training date range if specified
        if self.train_start_date is not None or self.train_end_date is not None:
            print(f"\n🎯 FILTERING TO TRAINING PERIOD:")
            
            if self.train_start_date is not None:
                before_filter = len(self.raw_df)
                self.raw_df = self.raw_df[self.raw_df['start_time'] >= self.train_start_date]
                print(f"  After start date filter: {len(self.raw_df):,} rows (removed {before_filter - len(self.raw_df):,})")
            
            if self.train_end_date is not None:
                before_filter = len(self.raw_df)
                self.raw_df = self.raw_df[self.raw_df['start_time'] <= self.train_end_date]
                print(f"  After end date filter: {len(self.raw_df):,} rows (removed {before_filter - len(self.raw_df):,})")
            
            print(f"✓ Training date range: {self.raw_df['start_time'].min()} to {self.raw_df['start_time'].max()}")
        
        # Sample data if specified (sample AFTER date filtering)
        if self.sample_size is not None:
            if isinstance(self.sample_size, float):
                n_samples = int(len(self.raw_df) * self.sample_size)
            else:
                n_samples = self.sample_size
            
            print(f"\nSampling {n_samples:,} rows for faster training...")
            self.raw_df = self.raw_df.sample(n=n_samples, random_state=self.random_state)
            print(f"Sampled data shape: {self.raw_df.shape}")
        
        # Preprocess if needed
        if self.needs_preprocessing:
            self.df = self.preprocess_data()
        else:
            self.df = self.raw_df
        
        return self.df
    
    def preprocess_data(self):
        """
        Transform raw trip data into aggregated station-level demand data
        with engineered features
        """
        print("\n" + "="*60)
        print("PREPROCESSING RAW TRIP DATA")
        print("="*60)
        
        df = self.raw_df.copy()
        
        # Ensure datetime columns
        if not pd.api.types.is_datetime64_any_dtype(df['start_time']):
            print("Converting start_time to datetime...")
            df['start_time'] = pd.to_datetime(df['start_time'])
        if not pd.api.types.is_datetime64_any_dtype(df['stop_time']):
            print("Converting stop_time to datetime...")
            df['stop_time'] = pd.to_datetime(df['stop_time'])
        
        # Step 1: Aggregate to hourly station-level pickups
        print("\nAggregating pickups by station and hour...")
        pickups = df.groupby([
            pd.Grouper(key='start_time', freq='1H'),
            'start_station_id'
        ]).agg({
            'ride_id': 'count',
            'start_station_latitude': 'first',
            'start_station_longitude': 'first',
            'user_type': lambda x: (x == 'member').sum() / len(x) if len(x) > 0 else 0,
        }).reset_index()
        
        pickups.columns = ['time', 'station_id', 'pickups', 'latitude', 
                          'longitude', 'member_ratio']
        
        # Step 2: Aggregate dropoffs by station and hour
        print("Aggregating dropoffs by station and hour...")
        dropoffs = df.groupby([
            pd.Grouper(key='stop_time', freq='1H'),
            'end_station_id'
        ]).agg({
            'ride_id': 'count'
        }).reset_index()
        
        dropoffs.columns = ['time', 'station_id', 'dropoffs']
        
        # Step 3: Merge pickups and dropoffs
        print("Merging pickups and dropoffs...")
        station_demand = pd.merge(
            pickups, 
            dropoffs, 
            on=['time', 'station_id'], 
            how='outer'
        ).fillna(0)
        
        # Calculate net flow
        station_demand['net_flow'] = (station_demand['dropoffs'] - 
                                      station_demand['pickups'])
        
        # Step 4: Engineer temporal features
        print("Engineering temporal features...")
        station_demand['hour'] = station_demand['time'].dt.hour
        station_demand['day_of_week'] = station_demand['time'].dt.dayofweek
        station_demand['month'] = station_demand['time'].dt.month
        station_demand['day_of_year'] = station_demand['time'].dt.dayofyear
        station_demand['week_of_year'] = station_demand['time'].dt.isocalendar().week
        
        # Binary features
        station_demand['is_weekend'] = (station_demand['day_of_week'] >= 5).astype(int)
        station_demand['is_rush_hour_morning'] = station_demand['hour'].isin([7, 8, 9]).astype(int)
        station_demand['is_rush_hour_evening'] = station_demand['hour'].isin([17, 18, 19]).astype(int)
        station_demand['is_business_hours'] = station_demand['hour'].between(9, 17).astype(int)
        station_demand['is_night'] = ((station_demand['hour'] >= 22) | 
                                      (station_demand['hour'] <= 5)).astype(int)
        
        # Cyclical encoding for hour and day_of_week
        station_demand['hour_sin'] = np.sin(2 * np.pi * station_demand['hour'] / 24)
        station_demand['hour_cos'] = np.cos(2 * np.pi * station_demand['hour'] / 24)
        station_demand['dow_sin'] = np.sin(2 * np.pi * station_demand['day_of_week'] / 7)
        station_demand['dow_cos'] = np.cos(2 * np.pi * station_demand['day_of_week'] / 7)
        
        # Step 5: Engineer lag features (CRITICAL for time series)
        print("Engineering lag features (this may take a moment)...")
        station_demand = station_demand.sort_values(['station_id', 'time'])
        
        # Create lag features for each station
        for lag in [1, 24, 168]:  # 1 hour, 1 day, 1 week
            station_demand[f'pickups_lag_{lag}h'] = station_demand.groupby('station_id')['pickups'].shift(lag)
            station_demand[f'dropoffs_lag_{lag}h'] = station_demand.groupby('station_id')['dropoffs'].shift(lag)
        
        # Rolling averages
        for window in [24, 168]:  # 1 day, 1 week
            station_demand[f'pickups_rolling_{window}h'] = (
                station_demand.groupby('station_id')['pickups']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
            station_demand[f'dropoffs_rolling_{window}h'] = (
                station_demand.groupby('station_id')['dropoffs']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
        
        # Step 6: Station-level features
        print("Engineering station-level features...")
        station_stats = station_demand.groupby('station_id').agg({
            'pickups': ['mean', 'std', 'max'],
            'dropoffs': ['mean', 'std', 'max'],
        })
        station_stats.columns = ['_'.join(col) for col in station_stats.columns]
        station_stats = station_stats.reset_index()
        
        station_demand = station_demand.merge(station_stats, on='station_id', how='left')
        
        # Step 7: Filter out very low activity periods
        print("Filtering low-activity periods...")
        initial_rows_filter = len(station_demand)
        
        # Keep only stations with at least some minimum activity
        station_total_activity = station_demand.groupby('station_id')[['pickups', 'dropoffs']].sum()
        active_stations = station_total_activity[
            (station_total_activity['pickups'] > 10) | 
            (station_total_activity['dropoffs'] > 10)
        ].index
        
        station_demand = station_demand[station_demand['station_id'].isin(active_stations)]
        filtered_rows = initial_rows_filter - len(station_demand)
        
        print(f"  Filtered out {filtered_rows:,} rows from inactive stations")
        
        # Drop rows with NaN (from lag features at the beginning)
        initial_rows = len(station_demand)
        station_demand = station_demand.dropna()
        dropped_rows = initial_rows - len(station_demand)
        
        print(f"\nPreprocessing complete!")
        print(f"  Original trips: {len(df):,}")
        print(f"  Aggregated to: {len(station_demand):,} station-hour records")
        print(f"  Active stations: {len(active_stations)}")
        print(f"  Dropped {dropped_rows:,} rows due to lag features")
        print(f"  Final features: {len(station_demand.columns)} columns")
        print(f"  Date range: {station_demand['time'].min()} to {station_demand['time'].max()}")
        
        return station_demand
    
    def prepare_features(self, target_col='pickups'):
        """
        Prepare features and target variable
        
        Parameters:
        -----------
        target_col : str
            Target variable name ('pickups', 'dropoffs', or 'net_flow')
        """
        print(f"\nPreparing features for {target_col} prediction...")
        
        # Check if target exists
        if target_col not in self.df.columns:
            raise ValueError(f"Target column '{target_col}' not found in data. "
                           f"Available columns: {list(self.df.columns)}")
        
        # Identify feature columns (exclude targets, identifiers, and time)
        exclude_cols = ['pickups', 'dropoffs', 'net_flow', 'ride_id', 
                       'start_time', 'stop_time', 'start_station_name', 
                       'end_station_name', 'time', 'station_id']
        
        self.feature_names = [col for col in self.df.columns 
                             if col not in exclude_cols and 
                             not self.df[col].isna().all()]
        
        X = self.df[self.feature_names]
        y = self.df[target_col]
        
        print(f"Features: {len(self.feature_names)}")
        print(f"Target: {target_col}")
        print(f"Target range: {y.min():.1f} to {y.max():.1f}")
        print(f"Target mean: {y.mean():.2f}")
        print(f"\nFeature categories:")
        print(f"  - Temporal: hour, day_of_week, month, etc.")
        print(f"  - Lag features: pickups_lag_*, dropoffs_lag_*")
        print(f"  - Rolling features: *_rolling_*")
        print(f"  - Station stats: *_mean, *_std, *_max")
        print(f"  - Location: latitude, longitude")
        
        return X, y
    
    def split_data(self, X, y, test_size=0.2, time_based=True):
        """
        Split data into train/test sets
        
        Parameters:
        -----------
        test_size : float
            Proportion of data for testing
        time_based : bool
            If True, use time-based split (HIGHLY recommended for time series)
            If False, use random split (NOT recommended)
        """
        print(f"\nSplitting data (test_size={test_size})...")
        
        if time_based and 'time' in self.df.columns:
            # Sort by time first
            sorted_df = self.df.sort_values('time').reset_index(drop=True)
            
            # Calculate split point
            split_idx = int(len(sorted_df) * (1 - test_size))
            
            # Get indices for train/test
            train_indices = sorted_df.index[:split_idx]
            test_indices = sorted_df.index[split_idx:]
            
            # Split using original indices
            X_train = X.iloc[train_indices]
            X_test = X.iloc[test_indices]
            y_train = y.iloc[train_indices]
            y_test = y.iloc[test_indices]
            
            # Get time ranges for reporting
            train_times = sorted_df.loc[train_indices, 'time']
            test_times = sorted_df.loc[test_indices, 'time']
            
            print(f"✓ Using time-based split (RECOMMENDED)")
            print(f"  Train period: {train_times.min()} to {train_times.max()}")
            print(f"  Test period:  {test_times.min()} to {test_times.max()}")
        else:
            # Random split
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=self.random_state
            )
            print("⚠️  Using random split (NOT recommended for time series)")
        
        print(f"Train set: {X_train.shape[0]:,} rows")
        print(f"Test set: {X_test.shape[0]:,} rows")
        
        return X_train, X_test, y_train, y_test
    
    def train_model(self, X_train, y_train, target_name='pickups', 
                   n_estimators=100, max_depth=20, min_samples_split=20,
                   min_samples_leaf=10, max_features='sqrt', n_jobs=-1):
        """
        Train Random Forest model with optimized parameters for large datasets
        """
        print(f"\n{'='*60}")
        print(f"Training Random Forest for {target_name}")
        print(f"{'='*60}")
        
        start_time = datetime.now()
        
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            n_jobs=n_jobs,
            random_state=self.random_state,
            verbose=1,
            warm_start=False,
            oob_score=True
        )
        
        print(f"\nModel parameters:")
        print(f"  - Trees: {n_estimators}")
        print(f"  - Max depth: {max_depth}")
        print(f"  - Min samples split: {min_samples_split}")
        print(f"  - Min samples leaf: {min_samples_leaf}")
        print(f"  - Max features: {max_features}")
        print(f"  - Parallel jobs: {n_jobs}")
        
        model.fit(X_train, y_train)
        
        training_time = (datetime.now() - start_time).total_seconds()
        print(f"\n✓ Training completed in {training_time:.2f} seconds")
        print(f"  OOB Score (R²): {model.oob_score_:.4f}")
        
        return model, training_time
    
    def evaluate_model(self, model, X_test, y_test, target_name='pickups'):
        """Evaluate model performance"""
        print(f"\nEvaluating {target_name} model...")
        
        # Predictions
        start_time = datetime.now()
        y_pred = model.predict(X_test)
        prediction_time = (datetime.now() - start_time).total_seconds()
        
        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1))) * 100
        
        results = {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'prediction_time': prediction_time,
            'predictions': y_pred,
            'actuals': y_test
        }
        
        print(f"\n{target_name.upper()} Model Performance:")
        print(f"{'='*50}")
        print(f"  MAE:  {mae:.2f}")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  R²:   {r2:.4f}")
        print(f"  MAPE: {mape:.2f}%")
        print(f"  Prediction time: {prediction_time:.2f}s for {len(X_test):,} samples")
        
        return results
    
    def plot_feature_importance(self, model, target_name='pickups', top_n=20):
        """Plot top N most important features"""
        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False).head(top_n)
        
        plt.figure(figsize=(10, 6))
        sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
        plt.title(f'Top {top_n} Feature Importances - {target_name}')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.savefig(f'feature_importance_{target_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"\n✓ Feature importance plot saved as 'feature_importance_{target_name}.png'")
        
        return importance_df
    
    def plot_predictions(self, results, target_name='pickups', n_samples=1000):
        """Plot actual vs predicted values"""
        if len(results['actuals']) > n_samples:
            indices = np.random.choice(len(results['actuals']), n_samples, replace=False)
            actuals_plot = results['actuals'].iloc[indices]
            predictions_plot = results['predictions'][indices]
        else:
            actuals_plot = results['actuals']
            predictions_plot = results['predictions']
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Scatter plot
        axes[0].scatter(actuals_plot, predictions_plot, alpha=0.5, s=10)
        axes[0].plot([actuals_plot.min(), actuals_plot.max()], 
                     [actuals_plot.min(), actuals_plot.max()], 
                     'r--', lw=2, label='Perfect prediction')
        axes[0].set_xlabel('Actual')
        axes[0].set_ylabel('Predicted')
        axes[0].set_title(f'Actual vs Predicted - {target_name}')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Residuals
        residuals = actuals_plot - predictions_plot
        axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
        axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
        axes[1].set_xlabel('Residual (Actual - Predicted)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title(f'Residuals Distribution - {target_name}')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'predictions_{target_name}.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Prediction plots saved as 'predictions_{target_name}.png'")
    
    def save_model(self, model, target_name='pickups'):
        """Save trained model"""
        filename = f'rf_model_{target_name}.joblib'
        joblib.dump(model, filename)
        print(f"\n✓ Model saved as '{filename}'")
    
    def run_full_pipeline(self, train_pickups=True, train_dropoffs=True):
        """
        Run complete pipeline for both pickups and dropoffs prediction
        """
        print("\n" + "="*60)
        print("BLUEBIKES DEMAND PREDICTION PIPELINE")
        print("="*60)
        
        # Load data
        self.load_data()
        
        # Train pickups model
        if train_pickups and 'pickups' in self.df.columns:
            X, y = self.prepare_features(target_col='pickups')
            X_train, X_test, y_train, y_test = self.split_data(X, y, time_based=True)
            
            self.pickup_model, train_time = self.train_model(
                X_train, y_train, target_name='pickups'
            )
            
            self.results['pickups'] = self.evaluate_model(
                self.pickup_model, X_test, y_test, target_name='pickups'
            )
            
            self.plot_feature_importance(self.pickup_model, target_name='pickups')
            self.plot_predictions(self.results['pickups'], target_name='pickups')
            self.save_model(self.pickup_model, target_name='pickups')
        
        # Train dropoffs model
        if train_dropoffs and 'dropoffs' in self.df.columns:
            X, y = self.prepare_features(target_col='dropoffs')
            X_train, X_test, y_train, y_test = self.split_data(X, y, time_based=True)
            
            self.dropoff_model, train_time = self.train_model(
                X_train, y_train, target_name='dropoffs'
            )
            
            self.results['dropoffs'] = self.evaluate_model(
                self.dropoff_model, X_test, y_test, target_name='dropoffs'
            )
            
            self.plot_feature_importance(self.dropoff_model, target_name='dropoffs')
            self.plot_predictions(self.results['dropoffs'], target_name='dropoffs')
            self.save_model(self.dropoff_model, target_name='dropoffs')
        
        # Calculate net flow if both models trained
        if train_pickups and train_dropoffs and self.pickup_model and self.dropoff_model:
            print("\n" + "="*60)
            print("NET FLOW ANALYSIS (Dropoffs - Pickups)")
            print("="*60)
            
            net_flow_pred = (self.results['dropoffs']['predictions'] - 
                           self.results['pickups']['predictions'])
            net_flow_actual = (self.results['dropoffs']['actuals'].values - 
                             self.results['pickups']['actuals'].values)
            
            mae_netflow = mean_absolute_error(net_flow_actual, net_flow_pred)
            rmse_netflow = np.sqrt(mean_squared_error(net_flow_actual, net_flow_pred))
            r2_netflow = r2_score(net_flow_actual, net_flow_pred)
            
            print(f"\nNet Flow Prediction Performance:")
            print(f"  MAE:  {mae_netflow:.2f}")
            print(f"  RMSE: {rmse_netflow:.2f}")
            print(f"  R²:   {r2_netflow:.4f}")
        
        print("\n" + "="*60)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*60)
        
        return self.results



In [5]:
# ========================================================================
# FOR RAW TRIP DATA (like yours with ride_id, start_time, etc.)
# ========================================================================
pipeline = BlueBikesRFPipeline(
    pickle_path='../data_pipeline/data/processed/bluebikes/after_duplicates.pkl',
    sample_size=0.8,  # Use 10% of data for faster processing
    random_state=42,
    needs_preprocessing=True,  # Set to True for raw trip data
    train_start_date='2023-12-31',  # Jan 1
    train_end_date='2025-06-30'     # June 30
)

# Run full pipeline
results = pipeline.run_full_pipeline(
    train_pickups=True,
    train_dropoffs=True
)

# Access results
print("\n" + "="*60)
print("SUMMARY OF RESULTS")
print("="*60)
for target, metrics in results.items():
    print(f"\n{target.upper()}:")
    print(f"  MAE: {metrics['mae']:.2f}")
    print(f"  RMSE: {metrics['rmse']:.2f}")
    print(f"  R²: {metrics['r2']:.4f}")


BLUEBIKES RANDOM FOREST PIPELINE
Data: ../data_pipeline/data/processed/bluebikes/after_duplicates.pkl
Sample size: 0.8
Training start: 2023-12-31
Training end: 2025-06-30


BLUEBIKES DEMAND PREDICTION PIPELINE
Loading data...
Original data shape: (2084559, 13)
Columns: ['ride_id', 'rideable_type', 'start_time', 'stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'user_type']
Full date range: 2024-01-01 00:05:08+00:00 to 2025-10-31 23:56:01.017000+00:00

🎯 FILTERING TO TRAINING PERIOD:
  After start date filter: 1,381,928 rows (removed 702,631)
  After end date filter: 831,372 rows (removed 550,556)
✓ Training date range: 2024-01-01 00:05:08+00:00 to 2025-06-29 23:56:08.608000+00:00

Sampling 665,097 rows for faster training...
Sampled data shape: (665097, 13)

PREPROCESSING RAW TRIP DATA

Aggregating pickups by station and hour...
Aggregating d

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.4s finished



✓ Training completed in 23.61 seconds
  OOB Score (R²): 0.7517

Evaluating pickups model...


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished



PICKUPS Model Performance:
  MAE:  0.21
  RMSE: 0.44
  R²:   0.7613
  MAPE: 8.09%
  Prediction time: 0.32s for 136,245 samples

✓ Feature importance plot saved as 'feature_importance_pickups.png'
✓ Prediction plots saved as 'predictions_pickups.png'

✓ Model saved as 'rf_model_pickups.joblib'

Preparing features for dropoffs prediction...
Features: 33
Target: dropoffs
Target range: 0.0 to 18.0
Target mean: 0.89

Feature categories:
  - Temporal: hour, day_of_week, month, etc.
  - Lag features: pickups_lag_*, dropoffs_lag_*
  - Rolling features: *_rolling_*
  - Station stats: *_mean, *_std, *_max
  - Location: latitude, longitude

Splitting data (test_size=0.2)...
✓ Using time-based split (RECOMMENDED)
  Train period: 2024-01-13 19:00:00+00:00 to 2025-05-18 15:00:00+00:00
  Test period:  2025-05-18 15:00:00+00:00 to 2025-06-30 07:00:00+00:00
Train set: 544,976 rows
Test set: 136,245 rows

Training Random Forest for dropoffs

Model parameters:
  - Trees: 100
  - Max depth: 20
  - Min sa

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   23.5s finished



✓ Training completed in 31.03 seconds
  OOB Score (R²): 0.3812

Evaluating dropoffs model...


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.3s finished



DROPOFFS Model Performance:
  MAE:  0.50
  RMSE: 0.73
  R²:   0.3539
  MAPE: 29.30%
  Prediction time: 0.42s for 136,245 samples

✓ Feature importance plot saved as 'feature_importance_dropoffs.png'
✓ Prediction plots saved as 'predictions_dropoffs.png'

✓ Model saved as 'rf_model_dropoffs.joblib'

NET FLOW ANALYSIS (Dropoffs - Pickups)

Net Flow Prediction Performance:
  MAE:  0.53
  RMSE: 0.82
  R²:   0.6187

PIPELINE COMPLETED SUCCESSFULLY!

SUMMARY OF RESULTS

PICKUPS:
  MAE: 0.21
  RMSE: 0.44
  R²: 0.7613

DROPOFFS:
  MAE: 0.50
  RMSE: 0.73
  R²: 0.3539


In [ ]:
import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from calendar import monthrange
import warnings
warnings.filterwarnings('ignore')

class ModelEvaluator:
    """
    Evaluate trained model on any specified month/year with proper handling to avoid data leakage
    """
    
    def __init__(self, 
                 model_path='rf_model_pickups.joblib',
                 data_pickle_path='../data_pipeline/data/processed/bluebikes/station_id_mapping.pkl',
                 eval_year=2025,
                 eval_month=10,
                 train_end_date=None,
                 train_sample_size=0.1):
        """
        Initialize evaluator
        
        Parameters:
        -----------
        model_path : str
            Path to saved model (.joblib file)
        data_pickle_path : str
            Path to raw data pickle file
        eval_year : int
            Year for evaluation (e.g., 2025)
        eval_month : int
            Month for evaluation (1-12)
        train_end_date : str
            Last date used for TRAINING THE MODEL (e.g., '2025-06-30')
            This is the cutoff date your model was trained up to
            IMPORTANT: Must match the train_end_date used in training!
        train_sample_size : float or int or None
            Sample size for historical data (for speed):
            - float (0.0-1.0): fraction of training data (e.g., 0.1 = 10%)
            - int: exact number of rows
            - None: use all training data (slow!)
            Evaluation month is ALWAYS evaluated in full (no sampling)
        """
        self.model_path = model_path
        self.data_pickle_path = data_pickle_path
        self.eval_year = eval_year
        self.eval_month = eval_month
        self.train_sample_size = train_sample_size
        
        # Calculate month boundaries (timezone-aware UTC to match data)
        self.test_start_date = pd.to_datetime(f'{eval_year}-{eval_month:02d}-01', utc=True)
        last_day = monthrange(eval_year, eval_month)[1]
        self.test_end_date = pd.to_datetime(f'{eval_year}-{eval_month:02d}-{last_day} 23:59:59', utc=True)
        
        # Set train_end_date (CRITICAL: must match training!)
        if train_end_date is None:
            # Default: one day before eval month starts
            self.train_end_date = self.test_start_date - pd.Timedelta(days=1)
            print(f"\n⚠️  WARNING: train_end_date not specified!")
            print(f"   Defaulting to: {self.train_end_date.date()}")
            print(f"   Make sure this matches your model training cutoff!\n")
        else:
            self.train_end_date = pd.to_datetime(train_end_date, utc=True)
        
        self.model = None
        self.raw_data = None
        self.results = {}
        
        # Month name for display
        month_names = ['January', 'February', 'March', 'April', 'May', 'June',
                      'July', 'August', 'September', 'October', 'November', 'December']
        self.month_name = month_names[eval_month - 1]
        
        print(f"\n{'='*70}")
        print(f"MODEL EVALUATION INITIALIZED - {self.month_name.upper()} {eval_year}")
        print(f"{'='*70}")
        print(f"Model: {model_path}")
        print(f"Data: {data_pickle_path}")
        print(f"Model trained up to: {self.train_end_date.date()}")
        print(f"Historical data sample: {train_sample_size if train_sample_size else 'ALL (no sampling)'}")
        print(f"Evaluation period: {self.test_start_date.date()} to {self.test_end_date.date()}")
        print(f"{'='*70}\n")
    
    def load_model(self):
        """Load saved model"""
        print("Loading saved model...")
        self.model = joblib.load(self.model_path)
        print(f"✓ Model loaded: {type(self.model).__name__}")
        print(f"  Features: {len(self.model.feature_names_in_)}")
        print(f"  Estimators: {self.model.n_estimators if hasattr(self.model, 'n_estimators') else 'N/A'}")
        return self.model
    
    def load_data(self):
        """Load raw data from pickle"""
        print("\nLoading raw data...")
        with open(self.data_pickle_path, 'rb') as f:
            self.raw_data = pickle.load(f)
        
        print(f"✓ Data loaded: {self.raw_data.shape[0]:,} rows")
        
        # Ensure datetime columns are timezone-aware (UTC)
        if not pd.api.types.is_datetime64_any_dtype(self.raw_data['start_time']):
            self.raw_data['start_time'] = pd.to_datetime(self.raw_data['start_time'], utc=True)
        elif self.raw_data['start_time'].dt.tz is None:
            # Add timezone if missing
            self.raw_data['start_time'] = self.raw_data['start_time'].dt.tz_localize('UTC')
            
        if not pd.api.types.is_datetime64_any_dtype(self.raw_data['stop_time']):
            self.raw_data['stop_time'] = pd.to_datetime(self.raw_data['stop_time'], utc=True)
        elif self.raw_data['stop_time'].dt.tz is None:
            # Add timezone if missing
            self.raw_data['stop_time'] = self.raw_data['stop_time'].dt.tz_localize('UTC')
        
        # Show date range
        print(f"  Date range: {self.raw_data['start_time'].min()} to {self.raw_data['start_time'].max()}")
        
        return self.raw_data
    
    def preprocess_data(self, df):
        """
        Preprocess raw trip data into station-hour aggregates with features
        """
        print(f"\nPreprocessing data ({len(df):,} trips)...")
        
        # Step 1: Aggregate pickups
        pickups = df.groupby([
            pd.Grouper(key='start_time', freq='1H'),
            'start_station_id'
        ]).agg({
            'ride_id': 'count',
            'start_station_latitude': 'first',
            'start_station_longitude': 'first',
            'user_type': lambda x: (x == 'member').sum() / len(x) if len(x) > 0 else 0,
        }).reset_index()
        
        pickups.columns = ['time', 'station_id', 'pickups', 'latitude', 
                          'longitude', 'member_ratio']
        
        # Step 2: Aggregate dropoffs
        dropoffs = df.groupby([
            pd.Grouper(key='stop_time', freq='1H'),
            'end_station_id'
        ]).agg({
            'ride_id': 'count'
        }).reset_index()
        
        dropoffs.columns = ['time', 'station_id', 'dropoffs']
        
        # Step 3: Merge
        station_demand = pd.merge(
            pickups, 
            dropoffs, 
            on=['time', 'station_id'], 
            how='outer'
        ).fillna(0)
        
        station_demand['net_flow'] = station_demand['dropoffs'] - station_demand['pickups']
        
        # Step 4: Temporal features
        station_demand['hour'] = station_demand['time'].dt.hour
        station_demand['day_of_week'] = station_demand['time'].dt.dayofweek
        station_demand['month'] = station_demand['time'].dt.month
        station_demand['day_of_year'] = station_demand['time'].dt.dayofyear
        station_demand['week_of_year'] = station_demand['time'].dt.isocalendar().week
        
        station_demand['is_weekend'] = (station_demand['day_of_week'] >= 5).astype(int)
        station_demand['is_rush_hour_morning'] = station_demand['hour'].isin([7, 8, 9]).astype(int)
        station_demand['is_rush_hour_evening'] = station_demand['hour'].isin([17, 18, 19]).astype(int)
        station_demand['is_business_hours'] = station_demand['hour'].between(9, 17).astype(int)
        station_demand['is_night'] = ((station_demand['hour'] >= 22) | 
                                      (station_demand['hour'] <= 5)).astype(int)
        
        station_demand['hour_sin'] = np.sin(2 * np.pi * station_demand['hour'] / 24)
        station_demand['hour_cos'] = np.cos(2 * np.pi * station_demand['hour'] / 24)
        station_demand['dow_sin'] = np.sin(2 * np.pi * station_demand['day_of_week'] / 7)
        station_demand['dow_cos'] = np.cos(2 * np.pi * station_demand['day_of_week'] / 7)
        
        # Step 5: Lag features
        station_demand = station_demand.sort_values(['station_id', 'time'])
        
        for lag in [1, 24, 168]:
            station_demand[f'pickups_lag_{lag}h'] = station_demand.groupby('station_id')['pickups'].shift(lag)
            station_demand[f'dropoffs_lag_{lag}h'] = station_demand.groupby('station_id')['dropoffs'].shift(lag)
        
        # Step 6: Rolling features
        for window in [24, 168]:
            station_demand[f'pickups_rolling_{window}h'] = (
                station_demand.groupby('station_id')['pickups']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
            station_demand[f'dropoffs_rolling_{window}h'] = (
                station_demand.groupby('station_id')['dropoffs']
                .transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
        
        # Step 7: Station statistics
        station_stats = station_demand.groupby('station_id').agg({
            'pickups': ['mean', 'std', 'max'],
            'dropoffs': ['mean', 'std', 'max'],
        })
        station_stats.columns = ['_'.join(col) for col in station_stats.columns]
        station_stats = station_stats.reset_index()
        
        station_demand = station_demand.merge(station_stats, on='station_id', how='left')
        
        # Step 8: Filter inactive stations
        station_total_activity = station_demand.groupby('station_id')[['pickups', 'dropoffs']].sum()
        active_stations = station_total_activity[
            (station_total_activity['pickups'] > 10) | 
            (station_total_activity['dropoffs'] > 10)
        ].index
        
        station_demand = station_demand[station_demand['station_id'].isin(active_stations)]
        
        # Drop NaN
        station_demand = station_demand.dropna()
        
        print(f"✓ Preprocessed: {len(station_demand):,} station-hour records")
        print(f"  Active stations: {len(active_stations)}")
        print(f"  Date range: {station_demand['time'].min()} to {station_demand['time'].max()}")
        
        return station_demand
    
    def prepare_evaluation_data(self):
        """
        Prepare evaluation data ensuring no data leakage
        CRITICAL: Lag features must use only pre-evaluation data!
        Uses sampling for training data (speed) but full evaluation month
        """
        print(f"\n{'='*70}")
        print("PREPARING EVALUATION DATA (AVOIDING DATA LEAKAGE)")
        print(f"{'='*70}")
        
        # Split raw data
        train_raw = self.raw_data[self.raw_data['start_time'] <= self.train_end_date].copy()
        test_raw = self.raw_data[
            (self.raw_data['start_time'] >= self.test_start_date) & 
            (self.raw_data['start_time'] <= self.test_end_date)
        ].copy()
        
        print(f"\nRaw data split:")
        print(f"  Training trips (full): {len(train_raw):,} (up to {self.train_end_date.date()})")
        print(f"  Evaluation trips: {len(test_raw):,} ({self.test_start_date.date()} to {self.test_end_date.date()})")
        
        if len(test_raw) == 0:
            raise ValueError(f"No data found for {self.month_name} {self.eval_year}! "
                           f"Your data range: {self.raw_data['start_time'].min()} to {self.raw_data['start_time'].max()}")
        
        # OPTIMIZATION: Sample training data for speed (but keep ALL evaluation data)
        if self.train_sample_size is not None:
            if isinstance(self.train_sample_size, float):
                n_samples = int(len(train_raw) * self.train_sample_size)
            else:
                n_samples = self.train_sample_size
            
            print(f"\n SPEED OPTIMIZATION: Sampling training data")
            print(f"  Training trips (sampled): {n_samples:,} ({self.train_sample_size*100 if isinstance(self.train_sample_size, float) else 'N/A'}%)")
            print(f"  Evaluation trips (full): {len(test_raw):,} (NO sampling for evaluation!)")
            
            # Sample training data randomly but maintain temporal distribution
            train_raw_sampled = train_raw.sample(n=min(n_samples, len(train_raw)), 
                                                 random_state=42)
        else:
            train_raw_sampled = train_raw
            print(f"\n  Using ALL training data (no sampling)")
        
        # CRITICAL: Combine sampled train + full test BEFORE preprocessing
        # This ensures lag features for evaluation month use historical data
        combined_raw = pd.concat([train_raw_sampled, test_raw], ignore_index=True)
        
        print(f"\nCombined for lag computation: {len(combined_raw):,} trips")
        print(f"  = {len(train_raw_sampled):,} training + {len(test_raw):,} evaluation")
        
        # Preprocess combined data
        combined_processed = self.preprocess_data(combined_raw)
        
        # Extract ONLY evaluation month rows
        eval_processed = combined_processed[
            (combined_processed['time'] >= self.test_start_date) &
            (combined_processed['time'] <= self.test_end_date)
        ].copy()
        
        print(f"\n✓ {self.month_name} data ready: {len(eval_processed):,} station-hour records")
        print(f"  Stations: {eval_processed['station_id'].nunique()}")
        print(f"  Hours covered: {(eval_processed['time'].max() - eval_processed['time'].min()).days * 24}")
        
        # Verify no data leakage in lag features
        print(f"\nVerifying lag features use only pre-{self.month_name} data...")
        sample_first = eval_processed[eval_processed['time'] == self.test_start_date].head(1)
        if len(sample_first) > 0:
            lag_168_val = sample_first['pickups_lag_168h'].values[0]
            lag_date = self.test_start_date - pd.Timedelta(days=7)
            print(f"  Example: {self.month_name} 1st lag_168h = {lag_168_val:.2f} "
                  f"(from {lag_date.date()}, 7 days prior) ✓")
        
        return eval_processed
    
    def compute_metrics(self, actuals, predictions):
        """Compute evaluation metrics"""
        mae = mean_absolute_error(actuals, predictions)
        rmse = np.sqrt(mean_squared_error(actuals, predictions))
        r2 = r2_score(actuals, predictions)
        mape = np.mean(np.abs((actuals - predictions) / (actuals + 1))) * 100
        
        return {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'n_samples': len(actuals)
        }
    
    def predict_and_evaluate(self, eval_data):
        """Make predictions and compute overall metrics"""
        print(f"\n{'='*70}")
        print("MAKING PREDICTIONS AND EVALUATING")
        print(f"{'='*70}")
        
        # Get features in correct order
        X = eval_data[self.model.feature_names_in_]
        
        #  AUTO-DETECT TARGET from model filename
        if 'dropoff' in self.model_path.lower():
            target_col = 'dropoffs'
        elif 'pickup' in self.model_path.lower():
            target_col = 'pickups'
        elif 'net_flow' in self.model_path.lower():
            target_col = 'net_flow'
        else:
            # Default to pickups with warning
            target_col = 'pickups'
            print("  Warning: Could not detect target from model filename, defaulting to 'pickups'")
        
        y_actual = eval_data[target_col]
        
        print(f"Target variable: {target_col}")
        print(f"Predicting on {len(X):,} station-hours...")
        y_pred = self.model.predict(X)
        
        # Create results dataframe
        results_df = pd.DataFrame({
            'time': eval_data['time'],
            'station_id': eval_data['station_id'],
            'hour': eval_data['hour'],
            'day_of_week': eval_data['day_of_week'],
            'actual': y_actual,
            'predicted': y_pred,
            'error': y_actual - y_pred,
            'abs_error': np.abs(y_actual - y_pred)
        })
            
        # Overall metrics
        overall_metrics = self.compute_metrics(y_actual, y_pred)
        
        print(f"\n{'='*70}")
        print("OVERALL METRICS")
        print(f"{'='*70}")
        print(f"Samples:      {overall_metrics['n_samples']:,}")
        print(f"MAE:          {overall_metrics['mae']:.3f}")
        print(f"RMSE:         {overall_metrics['rmse']:.3f}")
        print(f"R²:           {overall_metrics['r2']:.4f}")
        print(f"MAPE:         {overall_metrics['mape']:.2f}%")
        
        return results_df, overall_metrics
    
    def temporal_analysis(self, results_df):
        """Analyze performance by time dimensions"""
        print(f"\n{'='*70}")
        print("TEMPORAL ANALYSIS")
        print(f"{'='*70}")
        
        # By hour
        by_hour = results_df.groupby('hour').apply(
            lambda x: pd.Series(self.compute_metrics(x['actual'], x['predicted']))
        )
        
        print(f"\nPerformance by Hour of Day:")
        print(f"{'Hour':<6} {'MAE':<8} {'R²':<8} {'Samples':<10}")
        print("-" * 40)
        for hour in sorted(by_hour.index):
            metrics = by_hour.loc[hour]
            print(f"{hour:02d}:00  {metrics['mae']:6.3f}   {metrics['r2']:6.4f}   {int(metrics['n_samples']):,}")
        
        # By day of week
        by_dow = results_df.groupby('day_of_week').apply(
            lambda x: pd.Series(self.compute_metrics(x['actual'], x['predicted']))
        )
        
        dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        
        print(f"\nPerformance by Day of Week:")
        print(f"{'Day':<10} {'MAE':<8} {'R²':<8} {'Samples':<10}")
        print("-" * 40)
        for dow in sorted(by_dow.index):
            metrics = by_dow.loc[dow]
            print(f"{dow_names[dow]:<10} {metrics['mae']:6.3f}   {metrics['r2']:6.4f}   {int(metrics['n_samples']):,}")
        
        # By week
        results_df['week'] = results_df['time'].dt.isocalendar().week
        by_week = results_df.groupby('week').apply(
            lambda x: pd.Series(self.compute_metrics(x['actual'], x['predicted']))
        )
        
        print(f"\nPerformance by Week:")
        print(f"{'Week':<6} {'MAE':<8} {'R²':<8} {'Samples':<10}")
        print("-" * 40)
        for week in sorted(by_week.index):
            metrics = by_week.loc[week]
            print(f"{int(week):02d}     {metrics['mae']:6.3f}   {metrics['r2']:6.4f}   {int(metrics['n_samples']):,}")
        
        return {
            'by_hour': by_hour,
            'by_dow': by_dow,
            'by_week': by_week
        }
    
    def station_analysis(self, results_df):
        """Analyze performance by station"""
        print(f"\n{'='*70}")
        print("STATION ANALYSIS")
        print(f"{'='*70}")
        
        by_station = results_df.groupby('station_id').apply(
            lambda x: pd.Series({
                **self.compute_metrics(x['actual'], x['predicted']),
                'mean_actual': x['actual'].mean(),
                'mean_predicted': x['predicted'].mean(),
                'bias': x['predicted'].mean() - x['actual'].mean()
            })
        )
        
        # Worst stations
        worst_stations = by_station.nlargest(10, 'mae')
        print(f"\nWORST 10 STATIONS (Highest MAE):")
        print(f"{'Station':<10} {'MAE':<8} {'R²':<8} {'Bias':<8} {'Samples':<10}")
        print("-" * 50)
        for station_id in worst_stations.index:
            metrics = worst_stations.loc[station_id]
            print(f"{int(station_id):<10} {metrics['mae']:6.3f}   {metrics['r2']:6.4f}   "
                  f"{metrics['bias']:+6.3f}   {int(metrics['n_samples']):,}")
        
        # Best stations
        best_stations = by_station.nsmallest(10, 'mae')
        print(f"\nBEST 10 STATIONS (Lowest MAE):")
        print(f"{'Station':<10} {'MAE':<8} {'R²':<8} {'Bias':<8} {'Samples':<10}")
        print("-" * 50)
        for station_id in best_stations.index:
            metrics = best_stations.loc[station_id]
            print(f"{int(station_id):<10} {metrics['mae']:6.3f}   {metrics['r2']:6.4f}   "
                  f"{metrics['bias']:+6.3f}   {int(metrics['n_samples']):,}")
        
        return {
            'by_station': by_station,
            'worst': worst_stations,
            'best': best_stations
        }
    
    def create_visualizations(self, results_df):
        """Create comprehensive visualizations"""
        print(f"\n{'='*70}")
        print("CREATING VISUALIZATIONS")
        print(f"{'='*70}")
        
        fig = plt.figure(figsize=(16, 12))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # 1. Time series - Daily average
        ax1 = fig.add_subplot(gs[0, :])
        daily = results_df.groupby(results_df['time'].dt.date).agg({
            'actual': 'mean',
            'predicted': 'mean'
        })
        ax1.plot(daily.index, daily['actual'], label='Actual', linewidth=2, alpha=0.8)
        ax1.plot(daily.index, daily['predicted'], label='Predicted', 
                linewidth=2, linestyle='--', alpha=0.8)
        ax1.set_title(f'Daily Average Pickups per Station-Hour - {self.month_name} {self.eval_year}', 
                     fontsize=14, fontweight='bold')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Average Pickups')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
        
        # 2. Actual vs Predicted scatter
        ax2 = fig.add_subplot(gs[1, 0])
        sample_size = min(1000, len(results_df))
        sample = results_df.sample(n=sample_size, random_state=42)
        ax2.scatter(sample['actual'], sample['predicted'], alpha=0.5, s=10)
        max_val = max(sample['actual'].max(), sample['predicted'].max())
        ax2.plot([0, max_val], [0, max_val], 'r--', lw=2, label='Perfect prediction')
        ax2.set_xlabel('Actual')
        ax2.set_ylabel('Predicted')
        ax2.set_title('Actual vs Predicted', fontsize=12, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # 3. Residuals distribution
        ax3 = fig.add_subplot(gs[1, 1])
        ax3.hist(results_df['error'], bins=50, edgecolor='black', alpha=0.7)
        ax3.axvline(x=0, color='r', linestyle='--', lw=2)
        ax3.set_xlabel('Residual (Actual - Predicted)')
        ax3.set_ylabel('Frequency')
        ax3.set_title('Residuals Distribution', fontsize=12, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # 4. MAE by hour
        ax4 = fig.add_subplot(gs[1, 2])
        hourly_mae = results_df.groupby('hour')['abs_error'].mean()
        ax4.bar(hourly_mae.index, hourly_mae.values, color='steelblue', alpha=0.7)
        ax4.set_xlabel('Hour of Day')
        ax4.set_ylabel('MAE')
        ax4.set_title('MAE by Hour', fontsize=12, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # 5. MAE by day of week
        ax5 = fig.add_subplot(gs[2, 0])
        dow_mae = results_df.groupby('day_of_week')['abs_error'].mean()
        dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        ax5.bar(range(7), dow_mae.values, color='coral', alpha=0.7)
        ax5.set_xticks(range(7))
        ax5.set_xticklabels(dow_names)
        ax5.set_ylabel('MAE')
        ax5.set_title('MAE by Day of Week', fontsize=12, fontweight='bold')
        ax5.grid(True, alpha=0.3, axis='y')
        
        # 6. Error heatmap (hour x day of week)
        ax6 = fig.add_subplot(gs[2, 1:])
        error_pivot = results_df.pivot_table(
            values='abs_error',
            index='hour',
            columns='day_of_week',
            aggfunc='mean'
        )
        sns.heatmap(error_pivot, annot=False, fmt='.2f', cmap='YlOrRd', 
                   cbar_kws={'label': 'MAE'}, ax=ax6)
        ax6.set_xticklabels(dow_names)
        ax6.set_ylabel('Hour of Day')
        ax6.set_xlabel('Day of Week')
        ax6.set_title('MAE Heatmap (Hour × Day)', fontsize=12, fontweight='bold')
        
        plt.suptitle(f'{self.month_name} {self.eval_year} Model Evaluation', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        filename = f'evaluation_{self.eval_year}_{self.eval_month:02d}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Saved: {filename}")
        
        # Additional plot: Station performance distribution
        fig2, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Station MAE distribution
        station_mae = results_df.groupby('station_id')['abs_error'].mean()
        axes[0].hist(station_mae, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        axes[0].axvline(x=station_mae.median(), color='r', linestyle='--', 
                       lw=2, label=f'Median: {station_mae.median():.3f}')
        axes[0].set_xlabel('MAE per Station')
        axes[0].set_ylabel('Number of Stations')
        axes[0].set_title('Distribution of Station-Level MAE', fontsize=12, fontweight='bold')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Station R² distribution
        station_r2 = results_df.groupby('station_id').apply(
            lambda x: r2_score(x['actual'], x['predicted']) if len(x) > 1 else 0
        )
        axes[1].hist(station_r2, bins=30, edgecolor='black', alpha=0.7, color='coral')
        axes[1].axvline(x=station_r2.median(), color='r', linestyle='--', 
                       lw=2, label=f'Median: {station_r2.median():.3f}')
        axes[1].set_xlabel('R² per Station')
        axes[1].set_ylabel('Number of Stations')
        axes[1].set_title('Distribution of Station-Level R²', fontsize=12, fontweight='bold')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        filename2 = f'evaluation_{self.eval_year}_{self.eval_month:02d}_stations.png'
        plt.savefig(filename2, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Saved: {filename2}")
    
    def generate_summary_report(self, overall_metrics, temporal_metrics, station_metrics):
        """Generate text summary report"""
        print(f"\n{'='*70}")
        print("GENERATING SUMMARY REPORT")
        print(f"{'='*70}")
        
        report = f"""
{'='*70}
{self.month_name.upper()} {self.eval_year} MODEL EVALUATION SUMMARY REPORT
{'='*70}

Model: {self.model_path}
Evaluation Period: {self.test_start_date.date()} to {self.test_end_date.date()}
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

{'='*70}
1. OVERALL PERFORMANCE
{'='*70}

Total Predictions: {overall_metrics['n_samples']:,}
MAE:               {overall_metrics['mae']:.3f} pickups/hour
RMSE:              {overall_metrics['rmse']:.3f} pickups/hour
R²:                {overall_metrics['r2']:.4f}
MAPE:              {overall_metrics['mape']:.2f}%

Interpretation:
- On average, predictions are off by {overall_metrics['mae']:.2f} pickups per hour
- Model explains {overall_metrics['r2']*100:.1f}% of variance in demand
- Typical prediction error is {overall_metrics['mape']:.1f}%

{'='*70}
2. TEMPORAL PERFORMANCE
{'='*70}

Best Hours (Lowest MAE):
"""
        best_hours = temporal_metrics['by_hour'].nsmallest(3, 'mae')
        for hour in best_hours.index:
            report += f"  {hour:02d}:00 - MAE: {best_hours.loc[hour, 'mae']:.3f}, R²: {best_hours.loc[hour, 'r2']:.4f}\n"
        
        report += "\nWorst Hours (Highest MAE):\n"
        worst_hours = temporal_metrics['by_hour'].nlargest(3, 'mae')
        for hour in worst_hours.index:
            report += f"  {hour:02d}:00 - MAE: {worst_hours.loc[hour, 'mae']:.3f}, R²: {worst_hours.loc[hour, 'r2']:.4f}\n"
        
        dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        report += "\nPerformance by Day:\n"
        for dow in range(7):
            metrics = temporal_metrics['by_dow'].loc[dow]
            report += f"  {dow_names[dow]:<10} - MAE: {metrics['mae']:.3f}, R²: {metrics['r2']:.4f}\n"
        
        report += f"""
{'='*70}
3. STATION PERFORMANCE
{'='*70}

Total Stations Evaluated: {len(station_metrics['by_station'])}

Top 5 Best Stations (Lowest MAE):
"""
        best_5 = station_metrics['best'].head(5)
        for station_id in best_5.index:
            metrics = best_5.loc[station_id]
            report += f"  Station {int(station_id):>4} - MAE: {metrics['mae']:.3f}, R²: {metrics['r2']:.4f}\n"
        
        report += "\nTop 5 Worst Stations (Highest MAE):\n"
        worst_5 = station_metrics['worst'].head(5)
        for station_id in worst_5.index:
            metrics = worst_5.loc[station_id]
            report += f"  Station {int(station_id):>4} - MAE: {metrics['mae']:.3f}, R²: {metrics['r2']:.4f}\n"
        
        report += f"""
{'='*70}
4. KEY FINDINGS & RECOMMENDATIONS
{'='*70}

Performance Assessment:
"""
        if overall_metrics['r2'] > 0.80:
            report += "✓ EXCELLENT - Model performance exceeds industry standards\n"
        elif overall_metrics['r2'] > 0.70:
            report += "✓ GOOD - Model performance meets industry standards\n"
        elif overall_metrics['r2'] > 0.60:
            report += "⚠ FAIR - Model performance is acceptable but could be improved\n"
        else:
            report += "⚠ NEEDS IMPROVEMENT - Model performance below expectations\n"
        
        # Check for weekday vs weekend performance
        weekday_r2 = temporal_metrics['by_dow'].loc[0:4, 'r2'].mean()
        weekend_r2 = temporal_metrics['by_dow'].loc[5:6, 'r2'].mean()
        
        if weekday_r2 - weekend_r2 > 0.1:
            report += "\n⚠ Weekend Performance Issue:\n"
            report += f"  Weekday R²: {weekday_r2:.3f}\n"
            report += f"  Weekend R²: {weekend_r2:.3f}\n"
            report += "  Recommendation: Consider weekend-specific model or features\n"
        
        report += f"""
{'='*70}
EVALUATION COMPLETE
{'='*70}
        """
        
        # Save report to file
        filename = f'evaluation_{self.eval_year}_{self.eval_month:02d}_report.txt'
        with open(filename, 'w') as f:
            f.write(report)
        
        print(report)
        print(f"\n✓ Saved: {filename}")
    
    def run_full_evaluation(self):
        """Execute complete evaluation pipeline"""
        print(f"\n{'#'*70}")
        print(f"# STARTING FULL {self.month_name.upper()} {self.eval_year} EVALUATION")
        print(f"{'#'*70}\n")
        
        start_time = datetime.now()
        
        # Step 1: Load model and data
        self.load_model()
        self.load_data()
        
        # Step 2: Prepare evaluation data (avoiding data leakage)
        eval_data = self.prepare_evaluation_data()
        
        # Step 3: Make predictions and compute overall metrics
        results_df, overall_metrics = self.predict_and_evaluate(eval_data)
        
        # Step 4: Temporal analysis
        temporal_metrics = self.temporal_analysis(results_df)
        
        # Step 5: Station analysis
        station_metrics = self.station_analysis(results_df)
        
        # Step 6: Create visualizations
        self.create_visualizations(results_df)
        
        # Step 7: Generate summary report
        self.generate_summary_report(overall_metrics, temporal_metrics, station_metrics)
        
        # Store results
        self.results = {
            'overall': overall_metrics,
            'temporal': temporal_metrics,
            'stations': station_metrics,
            'predictions': results_df
        }
        
        elapsed = (datetime.now() - start_time).total_seconds()
        
        print(f"\n{'#'*70}")
        print(f"# EVALUATION COMPLETED IN {elapsed:.1f} SECONDS")
        print(f"{'#'*70}\n")
        
        return self.results


In [7]:
    
# Option 1: Evaluate October 2025 (default)
evaluator = ModelEvaluator(
    model_path='rf_model_pickups.joblib',
    data_pickle_path='../data_pipeline/data/processed/bluebikes/after_duplicates.pkl',
    eval_year=2025,
    eval_month=7,  # October
    train_sample_size=0.8,
    train_end_date='2025-06-30' 
)

results = evaluator.run_full_evaluation()

print("\n" + "="*70)
print("RESULTS STORED IN:")
print("="*70)
print("1. results['overall']     - Overall metrics dictionary")
print("2. results['temporal']    - Performance by hour/day/week")
print("3. results['stations']    - Performance by station")
print("4. results['predictions'] - Full predictions DataFrame")
print("\nFiles generated:")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}.png")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}_stations.png")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}_report.txt")
print("="*70)

# ========================================================================
# EXAMPLE: Access specific results
# ========================================================================

print("\n" + "="*70)
print("QUICK RESULTS ACCESS:")
print("="*70)
print(f"Overall R²: {results['overall']['r2']:.4f}")
print(f"Overall MAE: {results['overall']['mae']:.3f}")
print(f"\nBest performing hour: {results['temporal']['by_hour']['mae'].idxmin():02d}:00")
print(f"Worst performing hour: {results['temporal']['by_hour']['mae'].idxmax():02d}:00")
print(f"\nBest station: {int(results['stations']['best'].index[0])}")
print(f"Worst station: {int(results['stations']['worst'].index[0])}")
print("="*70)


MODEL EVALUATION INITIALIZED - JULY 2025
Model: rf_model_pickups.joblib
Data: ../data_pipeline/data/processed/bluebikes/after_duplicates.pkl
Model trained up to: 2025-06-30
Historical data sample: 0.8
Evaluation period: 2025-07-01 to 2025-07-31


######################################################################
# STARTING FULL JULY 2025 EVALUATION
######################################################################

Loading saved model...
✓ Model loaded: RandomForestRegressor
  Features: 33
  Estimators: 100

Loading raw data...
✓ Data loaded: 2,084,559 rows
  Date range: 2024-01-01 00:05:08+00:00 to 2025-10-31 23:56:01.017000+00:00

PREPARING EVALUATION DATA (AVOIDING DATA LEAKAGE)

Raw data split:
  Training trips (full): 831,372 (up to 2025-06-30)
  Evaluation trips: 134,574 (2025-07-01 to 2025-07-31)

⚡ SPEED OPTIMIZATION: Sampling training data
  Training trips (sampled): 665,097 (80.0%)
  Evaluation trips (full): 134,574 (NO sampling for evaluation!)

Combined for lag com

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished



OVERALL METRICS
Samples:      124,473
MAE:          0.325
RMSE:         0.630
R²:           0.7192
MAPE:         11.44%

TEMPORAL ANALYSIS

Performance by Hour of Day:
Hour   MAE      R²       Samples   
----------------------------------------
00:00   0.146   0.8057   2,393
01:00   0.162   0.7919   1,620
02:00   0.108   0.8458   1,023
03:00   0.082   0.9092   504
04:00   0.065   0.9422   420
05:00   0.093   0.8659   1,351
06:00   0.166   0.8048   3,239
07:00   0.307   0.6736   5,375
08:00   0.445   0.5985   6,987
09:00   0.308   0.6979   6,626
10:00   0.252   0.7472   5,777
11:00   0.262   0.7515   5,688
12:00   0.299   0.7360   6,482
13:00   0.303   0.7265   6,618
14:00   0.298   0.7451   6,803
15:00   0.336   0.7215   7,439
16:00   0.413   0.7135   8,531
17:00   0.498   0.6974   9,275
18:00   0.438   0.7142   8,920
19:00   0.367   0.7236   8,140
20:00   0.313   0.7230   6,924
21:00   0.272   0.7250   5,745
22:00   0.275   0.6168   4,929
23:00   0.190   0.7582   3,664

Performance b

In [12]:
evaluator = ModelEvaluator(
    model_path='rf_model_dropoffs.joblib',  # ← Dropoffs model
    data_pickle_path='../data_pipeline/data/processed/bluebikes/after_duplicates.pkl',
    eval_year=2025,
    eval_month=7,
    train_sample_size=0.8,
    train_end_date='2025-06-30' 
)

# ========================================================================
# RUN EVALUATION
# ========================================================================

results = evaluator.run_full_evaluation()

# ========================================================================
# ACCESS RESULTS
# ========================================================================

print("\n" + "="*70)
print("RESULTS STORED IN:")
print("="*70)
print("1. results['overall']     - Overall metrics dictionary")
print("2. results['temporal']    - Performance by hour/day/week")
print("3. results['stations']    - Performance by station")
print("4. results['predictions'] - Full predictions DataFrame")
print("\nFiles generated:")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}.png")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}_stations.png")
print(f"- evaluation_{evaluator.eval_year}_{evaluator.eval_month:02d}_report.txt")
print("="*70)

# ========================================================================
# EXAMPLE: Access specific results
# ========================================================================

print("\n" + "="*70)
print("QUICK RESULTS ACCESS:")
print("="*70)
print(f"Overall R²: {results['overall']['r2']:.4f}")
print(f"Overall MAE: {results['overall']['mae']:.3f}")
print(f"\nBest performing hour: {results['temporal']['by_hour']['mae'].idxmin():02d}:00")
print(f"Worst performing hour: {results['temporal']['by_hour']['mae'].idxmax():02d}:00")
print(f"\nBest station: {int(results['stations']['best'].index[0])}")
print(f"Worst station: {int(results['stations']['worst'].index[0])}")
print("="*70)


MODEL EVALUATION INITIALIZED - JULY 2025
Model: rf_model_dropoffs.joblib
Data: ../data_pipeline/data/processed/bluebikes/after_duplicates.pkl
Model trained up to: 2025-06-30
Historical data sample: 0.8
Evaluation period: 2025-07-01 to 2025-07-31


######################################################################
# STARTING FULL JULY 2025 EVALUATION
######################################################################

Loading saved model...
✓ Model loaded: RandomForestRegressor
  Features: 33
  Estimators: 100

Loading raw data...
✓ Data loaded: 2,084,559 rows
  Date range: 2024-01-01 00:05:08+00:00 to 2025-10-31 23:56:01.017000+00:00

PREPARING EVALUATION DATA (AVOIDING DATA LEAKAGE)

Raw data split:
  Training trips (full): 831,372 (up to 2025-06-30)
  Evaluation trips: 134,574 (2025-07-01 to 2025-07-31)

⚡ SPEED OPTIMIZATION: Sampling training data
  Training trips (sampled): 665,097 (80.0%)
  Evaluation trips (full): 134,574 (NO sampling for evaluation!)

Combined for lag co

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    3.4s finished



OVERALL METRICS
Samples:      124,473
MAE:          0.660
RMSE:         0.962
R²:           0.3584
MAPE:         35.91%

TEMPORAL ANALYSIS

Performance by Hour of Day:
Hour   MAE      R²       Samples   
----------------------------------------
00:00   0.370   0.3718   2,393
01:00   0.374   0.3512   1,620
02:00   0.320   0.4680   1,023
03:00   0.256   0.5528   504
04:00   0.214   0.6709   420
05:00   0.229   0.6678   1,351
06:00   0.413   0.3877   3,239
07:00   0.586   0.3363   5,375
08:00   0.809   0.3719   6,987
09:00   0.701   0.3796   6,626
10:00   0.598   0.3465   5,777
11:00   0.594   0.3157   5,688
12:00   0.634   0.3111   6,482
13:00   0.629   0.3055   6,618
14:00   0.643   0.3109   6,803
15:00   0.667   0.3112   7,439
16:00   0.739   0.3522   8,531
17:00   0.854   0.3803   9,275
18:00   0.837   0.3467   8,920
19:00   0.750   0.3164   8,140
20:00   0.684   0.2767   6,924
21:00   0.618   0.2364   5,745
22:00   0.558   0.2223   4,929
23:00   0.483   0.2873   3,664

Performance b

In [ ]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

class RebalancingOptimizer:
    """
    Complete bike rebalancing system using trained demand prediction models.
    Predicts future demand and optimizes truck routes to prevent stockouts.
    """
    
    def __init__(self, 
                 pickup_model_path='rf_model_pickups.joblib',
                 dropoff_model_path='rf_model_dropoffs.joblib',
                 station_capacity=20,
                 min_bikes_threshold=3,
                 max_bikes_threshold=17,
                 prediction_horizon=8):
        """
        Initialize rebalancing optimizer
        
        Parameters:
        -----------
        pickup_model_path : str
            Path to trained pickup prediction model
        dropoff_model_path : str
            Path to trained dropoff prediction model
        station_capacity : int
            Maximum bikes a station can hold
        min_bikes_threshold : int
            Minimum bikes before considered "low"
        max_bikes_threshold : int
            Maximum bikes before considered "full"
        prediction_horizon : int
            Number of hours to predict ahead (default: 8)
        """
        self.pickup_model = joblib.load(pickup_model_path)
        self.dropoff_model = joblib.load(dropoff_model_path)
        
        self.station_capacity = station_capacity
        self.min_bikes_threshold = min_bikes_threshold
        self.max_bikes_threshold = max_bikes_threshold
        self.prediction_horizon = prediction_horizon
        
        print(f"\n{'='*70}")
        print("REBALANCING OPTIMIZER INITIALIZED")
        print(f"{'='*70}")
        print(f"Pickup model loaded: {pickup_model_path}")
        print(f"Dropoff model loaded: {dropoff_model_path}")
        print(f"Station capacity: {station_capacity} bikes")
        print(f"Thresholds: Low={min_bikes_threshold}, High={max_bikes_threshold}")
        print(f"Prediction horizon: {prediction_horizon} hours")
        print(f"{'='*70}\n")
    
    def prepare_features_for_prediction(self, target_time, stations_data, historical_data):
        """
        Prepare features for future time prediction
        
        Parameters:
        -----------
        target_time : datetime
            Time to predict for
        stations_data : DataFrame
            Station information (id, lat, lon, etc.)
        historical_data : DataFrame
            Recent historical data for computing lag/rolling features
        
        Returns:
        --------
        DataFrame with features for all stations at target_time
        """
        features_list = []
        
        for _, station in stations_data.iterrows():
            station_id = station['station_id']
            
            # Get historical data for this station
            station_hist = historical_data[
                historical_data['station_id'] == station_id
            ].sort_values('time')
            
            # Temporal features
            hour = target_time.hour
            day_of_week = target_time.dayofweek
            month = target_time.month
            day_of_year = target_time.dayofyear
            week_of_year = target_time.isocalendar()[1]
            
            # Binary features
            is_weekend = 1 if day_of_week >= 5 else 0
            is_rush_hour_morning = 1 if hour in [7, 8, 9] else 0
            is_rush_hour_evening = 1 if hour in [17, 18, 19] else 0
            is_business_hours = 1 if 9 <= hour <= 17 else 0
            is_night = 1 if hour >= 22 or hour <= 5 else 0
            
            # Cyclical encoding
            hour_sin = np.sin(2 * np.pi * hour / 24)
            hour_cos = np.cos(2 * np.pi * hour / 24)
            dow_sin = np.sin(2 * np.pi * day_of_week / 7)
            dow_cos = np.cos(2 * np.pi * day_of_week / 7)
            
            # Lag features (get from historical data)
            try:
                pickups_lag_1h = station_hist.iloc[-1]['pickups'] if len(station_hist) > 0 else 0
                pickups_lag_24h = station_hist.iloc[-24]['pickups'] if len(station_hist) >= 24 else pickups_lag_1h
                pickups_lag_168h = station_hist.iloc[-168]['pickups'] if len(station_hist) >= 168 else pickups_lag_24h
                
                dropoffs_lag_1h = station_hist.iloc[-1]['dropoffs'] if len(station_hist) > 0 else 0
                dropoffs_lag_24h = station_hist.iloc[-24]['dropoffs'] if len(station_hist) >= 24 else dropoffs_lag_1h
                dropoffs_lag_168h = station_hist.iloc[-168]['dropoffs'] if len(station_hist) >= 168 else dropoffs_lag_24h
            except:
                pickups_lag_1h = pickups_lag_24h = pickups_lag_168h = 0
                dropoffs_lag_1h = dropoffs_lag_24h = dropoffs_lag_168h = 0
            
            # Rolling features
            if len(station_hist) >= 24:
                pickups_rolling_24h = station_hist.tail(24)['pickups'].mean()
                dropoffs_rolling_24h = station_hist.tail(24)['dropoffs'].mean()
            else:
                pickups_rolling_24h = pickups_lag_1h
                dropoffs_rolling_24h = dropoffs_lag_1h
            
            if len(station_hist) >= 168:
                pickups_rolling_168h = station_hist.tail(168)['pickups'].mean()
                dropoffs_rolling_168h = station_hist.tail(168)['dropoffs'].mean()
            else:
                pickups_rolling_168h = pickups_rolling_24h
                dropoffs_rolling_168h = dropoffs_rolling_24h
            
            # Station statistics
            pickups_mean = station.get('pickups_mean', station_hist['pickups'].mean() if len(station_hist) > 0 else 0)
            pickups_std = station.get('pickups_std', station_hist['pickups'].std() if len(station_hist) > 0 else 0)
            pickups_max = station.get('pickups_max', station_hist['pickups'].max() if len(station_hist) > 0 else 0)
            dropoffs_mean = station.get('dropoffs_mean', station_hist['dropoffs'].mean() if len(station_hist) > 0 else 0)
            dropoffs_std = station.get('dropoffs_std', station_hist['dropoffs'].std() if len(station_hist) > 0 else 0)
            dropoffs_max = station.get('dropoffs_max', station_hist['dropoffs'].max() if len(station_hist) > 0 else 0)
            
            # Compile features
            features = {
                'latitude': station['latitude'],
                'longitude': station['longitude'],
                'member_ratio': station.get('member_ratio', 0.5),
                'hour': hour,
                'day_of_week': day_of_week,
                'month': month,
                'day_of_year': day_of_year,
                'week_of_year': week_of_year,
                'is_weekend': is_weekend,
                'is_rush_hour_morning': is_rush_hour_morning,
                'is_rush_hour_evening': is_rush_hour_evening,
                'is_business_hours': is_business_hours,
                'is_night': is_night,
                'hour_sin': hour_sin,
                'hour_cos': hour_cos,
                'dow_sin': dow_sin,
                'dow_cos': dow_cos,
                'pickups_lag_1h': pickups_lag_1h,
                'pickups_lag_24h': pickups_lag_24h,
                'pickups_lag_168h': pickups_lag_168h,
                'dropoffs_lag_1h': dropoffs_lag_1h,
                'dropoffs_lag_24h': dropoffs_lag_24h,
                'dropoffs_lag_168h': dropoffs_lag_168h,
                'pickups_rolling_24h': pickups_rolling_24h,
                'pickups_rolling_168h': pickups_rolling_168h,
                'dropoffs_rolling_24h': dropoffs_rolling_24h,
                'dropoffs_rolling_168h': dropoffs_rolling_168h,
                'pickups_mean': pickups_mean,
                'pickups_std': pickups_std,
                'pickups_max': pickups_max,
                'dropoffs_mean': dropoffs_mean,
                'dropoffs_std': dropoffs_std,
                'dropoffs_max': dropoffs_max,
            }
            
            features_list.append(features)
        
        return pd.DataFrame(features_list)
    
    def predict_demand(self, target_time, stations_data, historical_data):
        """
        Predict pickups and dropoffs for all stations at target time
        
        Returns:
        --------
        DataFrame with predictions for each station
        """
        # Prepare features
        features_df = self.prepare_features_for_prediction(target_time, stations_data, historical_data)
        
        # Ensure feature order matches model training
        X_pickups = features_df[self.pickup_model.feature_names_in_]
        X_dropoffs = features_df[self.dropoff_model.feature_names_in_]
        
        # Predict
        pickups_pred = self.pickup_model.predict(X_pickups)
        dropoffs_pred = self.dropoff_model.predict(X_dropoffs)
        
        # Create predictions dataframe
        predictions = pd.DataFrame({
            'station_id': stations_data['station_id'],
            'time': target_time,
            'predicted_pickups': pickups_pred,
            'predicted_dropoffs': dropoffs_pred,
            'predicted_net_flow': dropoffs_pred - pickups_pred,
            'latitude': stations_data['latitude'],
            'longitude': stations_data['longitude']
        })
        
        return predictions
    
    def predict_next_hours(self, current_time, stations_data, historical_data, hours=None):
        """
        Predict demand for next N hours
        
        Parameters:
        -----------
        current_time : datetime
            Current time
        stations_data : DataFrame
            Station information
        historical_data : DataFrame
            Recent historical aggregated data
        hours : int, optional
            Number of hours to predict (default: self.prediction_horizon)
        
        Returns:
        --------
        DataFrame with predictions for all stations and hours
        """
        if hours is None:
            hours = self.prediction_horizon
        
        print(f"\nPredicting demand for next {hours} hours...")
        print(f"Starting from: {current_time}")
        
        all_predictions = []
        
        for hour_offset in range(1, hours + 1):
            target_time = current_time + timedelta(hours=hour_offset)
            
            # Predict for this hour
            hour_predictions = self.predict_demand(target_time, stations_data, historical_data)
            all_predictions.append(hour_predictions)
            
            # Update historical data with prediction (for next iteration's lag features)
            # This is a simplification - in production, use actual data as it becomes available
            new_record = hour_predictions[['station_id', 'time', 'predicted_pickups', 'predicted_dropoffs']].copy()
            new_record.columns = ['station_id', 'time', 'pickups', 'dropoffs']
            historical_data = pd.concat([historical_data, new_record], ignore_index=True)
        
        predictions_df = pd.concat(all_predictions, ignore_index=True)
        
        print(f"✓ Predictions complete: {len(predictions_df):,} station-hour predictions")
        
        return predictions_df
    
    def simulate_inventory(self, predictions, current_inventory):
        """
        Simulate bike inventory over prediction horizon
        
        Parameters:
        -----------
        predictions : DataFrame
            Predicted pickups/dropoffs for all stations and hours
        current_inventory : dict
            Current bike count at each station {station_id: count}
        
        Returns:
        --------
        DataFrame with projected inventory over time
        """
        print("\nSimulating inventory over time...")
        
        inventory_timeline = []
        
        # Group by station
        for station_id in predictions['station_id'].unique():
            station_preds = predictions[predictions['station_id'] == station_id].sort_values('time')
            
            # Initialize inventory
            current_bikes = current_inventory.get(station_id, 10)  # Default: 10 bikes
            
            for _, row in station_preds.iterrows():
                # Update inventory based on predicted net flow
                current_bikes += row['predicted_net_flow']
                
                # Clip to capacity
                current_bikes = max(0, min(current_bikes, self.station_capacity))
                
                inventory_timeline.append({
                    'station_id': station_id,
                    'time': row['time'],
                    'predicted_bikes': current_bikes,
                    'predicted_pickups': row['predicted_pickups'],
                    'predicted_dropoffs': row['predicted_dropoffs'],
                    'predicted_net_flow': row['predicted_net_flow']
                })
        
        inventory_df = pd.DataFrame(inventory_timeline)
        
        print(f"✓ Inventory simulation complete")
        
        return inventory_df
    
    def identify_rebalancing_needs(self, inventory_timeline):
        """
        Identify stations that will need rebalancing
        
        Parameters:
        -----------
        inventory_timeline : DataFrame
            Projected inventory from simulate_inventory()
        
        Returns:
        --------
        DataFrame with rebalancing needs
        """
        print("\nIdentifying rebalancing needs...")
        
        needs = []
        
        # Group by station
        for station_id in inventory_timeline['station_id'].unique():
            station_timeline = inventory_timeline[
                inventory_timeline['station_id'] == station_id
            ].sort_values('time')
            
            # Check each hour
            for idx, row in station_timeline.iterrows():
                bikes = row['predicted_bikes']
                time = row['time']
                
                # Check if station will run low on bikes
                if bikes < self.min_bikes_threshold:
                    deficit = self.min_bikes_threshold + 2 - bikes  # Add buffer
                    
                    needs.append({
                        'station_id': station_id,
                        'time': time,
                        'action': 'ADD',
                        'bikes_needed': int(np.ceil(deficit)),
                        'current_predicted_bikes': bikes,
                        'priority': 'CRITICAL' if bikes <= 0 else 'HIGH' if bikes <= 1 else 'MEDIUM',
                        'hours_until': (time - inventory_timeline['time'].min()).total_seconds() / 3600
                    })
                    break  # Only flag first occurrence for this station
                
                # Check if station will become too full
                elif bikes > self.max_bikes_threshold:
                    surplus = bikes - (self.max_bikes_threshold - 2)  # Leave buffer
                    
                    needs.append({
                        'station_id': station_id,
                        'time': time,
                        'action': 'REMOVE',
                        'bikes_needed': int(np.ceil(surplus)),
                        'current_predicted_bikes': bikes,
                        'priority': 'CRITICAL' if bikes >= self.station_capacity else 'HIGH' if bikes >= 19 else 'MEDIUM',
                        'hours_until': (time - inventory_timeline['time'].min()).total_seconds() / 3600
                    })
                    break  # Only flag first occurrence
        
        needs_df = pd.DataFrame(needs)
        
        if len(needs_df) > 0:
            print(f"✓ Identified {len(needs_df)} stations needing rebalancing:")
            print(f"  - ADD bikes: {len(needs_df[needs_df['action'] == 'ADD'])}")
            print(f"  - REMOVE bikes: {len(needs_df[needs_df['action'] == 'REMOVE'])}")
            print(f"  - CRITICAL priority: {len(needs_df[needs_df['priority'] == 'CRITICAL'])}")
            print(f"  - HIGH priority: {len(needs_df[needs_df['priority'] == 'HIGH'])}")
            print(f"  - MEDIUM priority: {len(needs_df[needs_df['priority'] == 'MEDIUM'])}")
        else:
            print("✓ No rebalancing needed - all stations within acceptable range")
        
        return needs_df
    
    def optimize_routes(self, rebalancing_needs, stations_data, depot_location=None):
        """
        Optimize truck routes to fulfill rebalancing needs
        Uses Hungarian algorithm to match surplus and deficit stations
        
        Parameters:
        -----------
        rebalancing_needs : DataFrame
            Output from identify_rebalancing_needs()
        stations_data : DataFrame
            Station information including coordinates
        depot_location : tuple, optional
            (lat, lon) of truck depot. If None, uses centroid of stations
        
        Returns:
        --------
        DataFrame with optimized rebalancing routes
        """
        print("\nOptimizing rebalancing routes...")
        
        if len(rebalancing_needs) == 0:
            print("✓ No routes needed (no rebalancing required)")
            return pd.DataFrame()
        
        # Merge station coordinates
        needs_with_coords = rebalancing_needs.merge(
            stations_data[['station_id', 'latitude', 'longitude']],
            on='station_id',
            how='left'
        )
        
        # Separate surplus and deficit stations
        surplus_stations = needs_with_coords[needs_with_coords['action'] == 'REMOVE'].copy()
        deficit_stations = needs_with_coords[needs_with_coords['action'] == 'ADD'].copy()
        
        print(f"  Surplus stations: {len(surplus_stations)}")
        print(f"  Deficit stations: {len(deficit_stations)}")
        
        if len(surplus_stations) == 0 or len(deficit_stations) == 0:
            print("⚠️  Only one-way rebalancing needed (only surplus or only deficit)")
            # Handle one-way rebalancing
            return self._handle_one_way_rebalancing(surplus_stations, deficit_stations, depot_location)
        
        # Calculate distance matrix
        surplus_coords = surplus_stations[['latitude', 'longitude']].values
        deficit_coords = deficit_stations[['latitude', 'longitude']].values
        
        # Haversine distance in km
        distance_matrix = self._calculate_distance_matrix(surplus_coords, deficit_coords)
        
        # Solve assignment problem (minimize total distance)
        row_ind, col_ind = linear_sum_assignment(distance_matrix)
        
        # Create routes
        routes = []
        for i, j in zip(row_ind, col_ind):
            surplus_row = surplus_stations.iloc[i]
            deficit_row = deficit_stations.iloc[j]
            
            # Bikes to transfer (minimum of what's available and what's needed)
            bikes_to_move = min(surplus_row['bikes_needed'], deficit_row['bikes_needed'])
            
            routes.append({
                'route_id': len(routes) + 1,
                'from_station': int(surplus_row['station_id']),
                'to_station': int(deficit_row['station_id']),
                'bikes_to_move': int(bikes_to_move),
                'distance_km': distance_matrix[i, j],
                'from_priority': surplus_row['priority'],
                'to_priority': deficit_row['priority'],
                'priority': max(surplus_row['priority'], deficit_row['priority']),
                'from_lat': surplus_row['latitude'],
                'from_lon': surplus_row['longitude'],
                'to_lat': deficit_row['latitude'],
                'to_lon': deficit_row['longitude'],
                'deadline': min(surplus_row['time'], deficit_row['time'])
            })
        
        routes_df = pd.DataFrame(routes)
        
        # Sort by priority and distance
        priority_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2}
        routes_df['priority_rank'] = routes_df['priority'].map(priority_order)
        routes_df = routes_df.sort_values(['priority_rank', 'distance_km'])
        routes_df = routes_df.drop('priority_rank', axis=1)
        
        # Calculate total metrics
        total_distance = routes_df['distance_km'].sum()
        total_bikes = routes_df['bikes_to_move'].sum()
        
        print(f"\n✓ Routes optimized:")
        print(f"  Total routes: {len(routes_df)}")
        print(f"  Total bikes to move: {total_bikes}")
        print(f"  Total distance: {total_distance:.2f} km")
        print(f"  Average distance per route: {total_distance / len(routes_df):.2f} km")
        
        return routes_df
    
    def _calculate_distance_matrix(self, coords1, coords2):
        """
        Calculate Haversine distance matrix between two sets of coordinates
        
        Parameters:
        -----------
        coords1 : array-like, shape (n, 2)
            First set of (lat, lon) coordinates
        coords2 : array-like, shape (m, 2)
            Second set of (lat, lon) coordinates
        
        Returns:
        --------
        array, shape (n, m)
            Distance matrix in kilometers
        """
        # Convert to radians
        coords1_rad = np.radians(coords1)
        coords2_rad = np.radians(coords2)
        
        # Haversine formula
        distances = np.zeros((len(coords1), len(coords2)))
        
        for i, (lat1, lon1) in enumerate(coords1_rad):
            for j, (lat2, lon2) in enumerate(coords2_rad):
                dlat = lat2 - lat1
                dlon = lon2 - lon1
                
                a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                c = 2 * np.arcsin(np.sqrt(a))
                
                # Earth radius in km
                distances[i, j] = 6371 * c
        
        return distances
    
    def _handle_one_way_rebalancing(self, surplus_stations, deficit_stations, depot_location):
        """
        Handle case where only surplus OR only deficit exists
        (Bikes must come from/go to depot)
        """
        if depot_location is None:
            # Use centroid as default depot
            if len(surplus_stations) > 0:
                depot_location = (
                    surplus_stations['latitude'].mean(),
                    surplus_stations['longitude'].mean()
                )
            else:
                depot_location = (
                    deficit_stations['latitude'].mean(),
                    deficit_stations['longitude'].mean()
                )
        
        routes = []
        
        # If only surplus (bring bikes to depot)
        if len(surplus_stations) > 0 and len(deficit_stations) == 0:
            for _, station in surplus_stations.iterrows():
                distance = self._haversine_distance(
                    station['latitude'], station['longitude'],
                    depot_location[0], depot_location[1]
                )
                
                routes.append({
                    'route_id': len(routes) + 1,
                    'from_station': int(station['station_id']),
                    'to_station': 'DEPOT',
                    'bikes_to_move': int(station['bikes_needed']),
                    'distance_km': distance,
                    'priority': station['priority']
                })
        
        # If only deficit (bring bikes from depot)
        elif len(deficit_stations) > 0 and len(surplus_stations) == 0:
            for _, station in deficit_stations.iterrows():
                distance = self._haversine_distance(
                    depot_location[0], depot_location[1],
                    station['latitude'], station['longitude']
                )
                
                routes.append({
                    'route_id': len(routes) + 1,
                    'from_station': 'DEPOT',
                    'to_station': int(station['station_id']),
                    'bikes_to_move': int(station['bikes_needed']),
                    'distance_km': distance,
                    'priority': station['priority']
                })
        
        return pd.DataFrame(routes)
    
    def _haversine_distance(self, lat1, lon1, lat2, lon2):
        """Calculate Haversine distance between two points in km"""
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        return 6371 * c
    
    def generate_rebalancing_plan(self, current_time, stations_data, historical_data, 
                                  current_inventory, hours=None):
        """
        Complete rebalancing plan generation
        
        This is the MAIN METHOD to call for generating rebalancing instructions
        
        Parameters:
        -----------
        current_time : datetime
            Current time (now)
        stations_data : DataFrame
            Station information (id, lat, lon, capacity, etc.)
        historical_data : DataFrame
            Recent hourly aggregated data for computing lag features
        current_inventory : dict
            Current bike count at each station {station_id: bikes_count}
        hours : int, optional
            Prediction horizon (default: 8)
        
        Returns:
        --------
        dict with:
            - predictions: Future demand predictions
            - inventory: Projected inventory timeline
            - needs: Rebalancing needs identified
            - routes: Optimized truck routes
        """
        print(f"\n{'#'*70}")
        print(f"# GENERATING REBALANCING PLAN")
        print(f"# Current time: {current_time}")
        print(f"{'#'*70}\n")
        
        # Step 1: Predict demand for next N hours
        predictions = self.predict_next_hours(
            current_time, stations_data, historical_data, hours
        )
        
        # Step 2: Simulate inventory
        inventory_timeline = self.simulate_inventory(predictions, current_inventory)
        
        # Step 3: Identify rebalancing needs
        rebalancing_needs = self.identify_rebalancing_needs(inventory_timeline)
        
        # Step 4: Optimize routes
        if len(rebalancing_needs) > 0:
            routes = self.optimize_routes(rebalancing_needs, stations_data)
        else:
            routes = pd.DataFrame()
        
        # Package results
        plan = {
            'predictions': predictions,
            'inventory': inventory_timeline,
            'needs': rebalancing_needs,
            'routes': routes,
            'generated_at': current_time
        }
        
        self._print_summary(plan)
        
        return plan
    
    def _print_summary(self, plan):
        """Print summary of rebalancing plan"""
        print(f"\n{'='*70}")
        print("REBALANCING PLAN SUMMARY")
        print(f"{'='*70}")
        
        # Predictions summary
        total_predicted_pickups = plan['predictions']['predicted_pickups'].sum()
        total_predicted_dropoffs = plan['predictions']['predicted_dropoffs'].sum()
        
        print(f"\nDemand Forecast (next {self.prediction_horizon} hours):")
        print(f"  Total pickups: {total_predicted_pickups:.0f}")
        print(f"  Total dropoffs: {total_predicted_dropoffs:.0f}")
        print(f"  Net system flow: {total_predicted_dropoffs - total_predicted_pickups:+.0f}")
        
        # Inventory summary
        stations_low = len(plan['inventory'][plan['inventory']['predicted_bikes'] < self.min_bikes_threshold])
        stations_full = len(plan['inventory'][plan['inventory']['predicted_bikes'] > self.max_bikes_threshold])
        
        print(f"\nInventory Projections:")
        print(f"  Stations going low (< {self.min_bikes_threshold} bikes): {stations_low}")
        print(f"  Stations going full (> {self.max_bikes_threshold} bikes): {stations_full}")
        
        # Rebalancing summary
        if len(plan['needs']) > 0:
            print(f"\nRebalancing Needs:")
            print(f"  Stations requiring action: {len(plan['needs'])}")
            print(f"  Total bikes to move: {plan['needs']['bikes_needed'].sum():.0f}")
            
            if len(plan['routes']) > 0:
                print(f"\nOptimized Routes:")
                print(f"  Number of routes: {len(plan['routes'])}")
                print(f"  Total distance: {plan['routes']['distance_km'].sum():.2f} km")
                print(f"  Estimated time: {len(plan['routes']) * 20:.0f} minutes " 
                      f"(~20 min per route)")
        else:
            print(f"\n✓ NO REBALANCING NEEDED - All stations projected to stay within range")
        
        print(f"{'='*70}\n")
    
    def save_plan(self, plan, filename=None):
        """Save rebalancing plan to files"""
        if filename is None:
            timestamp = plan['generated_at'].strftime('%Y%m%d_%H%M')
            filename = f'rebalancing_plan_{timestamp}'
        
        # Save predictions
        plan['predictions'].to_csv(f'{filename}_predictions.csv', index=False)
        
        # Save inventory timeline
        plan['inventory'].to_csv(f'{filename}_inventory.csv', index=False)
        
        # Save needs
        if len(plan['needs']) > 0:
            plan['needs'].to_csv(f'{filename}_needs.csv', index=False)
        
        # Save routes
        if len(plan['routes']) > 0:
            plan['routes'].to_csv(f'{filename}_routes.csv', index=False)
        
        print(f"✓ Rebalancing plan saved:")
        print(f"  - {filename}_predictions.csv")
        print(f"  - {filename}_inventory.csv")
        if len(plan['needs']) > 0:
            print(f"  - {filename}_needs.csv")
        if len(plan['routes']) > 0:
            print(f"  - {filename}_routes.csv")




In [ ]:
# ========================================================================
# SETUP: Load necessary data
# ========================================================================

# Initialize optimizer
optimizer = RebalancingOptimizer(
    pickup_model_path='rf_model_pickups.joblib',
    dropoff_model_path='rf_model_dropoffs.joblib',
    station_capacity=20,
    min_bikes_threshold=3,
    max_bikes_threshold=17,
    prediction_horizon=8
)

# ========================================================================
# PREPARE DATA
# ========================================================================

# Example: Load station data
# In production, this comes from your database
stations_data = pd.DataFrame({
    'station_id': [1, 27, 28, 39, 68, 182, 204, 426, 530],
    'latitude': [42.3601, 42.3736, 42.3519, 42.3612, 42.3513, 42.3594, 42.3513, 42.3589, 42.3625],
    'longitude': [-71.0589, -71.1097, -71.0629, -71.0812, -71.0644, -71.0637, -71.0956, -71.0707, -71.0598],
    'member_ratio': [0.85, 0.78, 0.82, 0.75, 0.80, 0.88, 0.73, 0.81, 0.84],
    'pickups_mean': [3.2, 2.8, 3.5, 2.1, 2.9, 4.2, 2.5, 3.1, 3.8],
    'pickups_std': [4.5, 3.9, 4.8, 3.2, 4.1, 5.2, 3.7, 4.3, 4.9],
    'pickups_max': [24, 18, 22, 15, 20, 28, 16, 21, 25],
    'dropoffs_mean': [2.8, 2.5, 3.1, 1.9, 2.6, 3.8, 2.3, 2.8, 3.5],
    'dropoffs_std': [4.1, 3.5, 4.5, 2.9, 3.8, 4.8, 3.4, 4.0, 4.6],
    'dropoffs_max': [22, 16, 20, 14, 18, 25, 15, 19, 23]
})

# Example: Recent historical data (last 168 hours)
# In production, this comes from your real-time data pipeline
# For demo, create synthetic data
historical_data = pd.DataFrame({
    'station_id': np.repeat(stations_data['station_id'].values, 168),
    'time': pd.date_range(
        start=datetime.now() - timedelta(hours=168),
        end=datetime.now(),
        freq='1H'
    ).repeat(len(stations_data)),
    'pickups': np.random.poisson(lam=2, size=len(stations_data)*168),
    'dropoffs': np.random.poisson(lam=2, size=len(stations_data)*168)
})

# Example: Current inventory (bikes at each station right now)
# In production, this comes from real-time station sensors
current_inventory = {
    1: 5, 27: 12, 28: 8, 39: 15, 68: 3,
    182: 18, 204: 7, 426: 2, 530: 19
}

# ========================================================================
# GENERATE REBALANCING PLAN
# ========================================================================

current_time = datetime.now()

plan = optimizer.generate_rebalancing_plan(
    current_time=current_time,
    stations_data=stations_data,
    historical_data=historical_data,
    current_inventory=current_inventory,
    hours=8  # Predict next 8 hours
)

# ========================================================================
# VIEW RESULTS
# ========================================================================

print("\n" + "="*70)
print("DETAILED RESULTS")
print("="*70)

# Show predictions for next 4 hours
print("\nDemand Predictions (next 4 hours):")
print(plan['predictions'].head(4 * len(stations_data)))

# Show critical rebalancing needs
if len(plan['needs']) > 0:
    print("\nCritical Rebalancing Needs:")
    critical = plan['needs'][plan['needs']['priority'] == 'CRITICAL']
    if len(critical) > 0:
        print(critical[['station_id', 'action', 'bikes_needed', 'time', 'hours_until']])
    else:
        print("No CRITICAL needs")
    
    # Show recommended routes
    if len(plan['routes']) > 0:
        print("\nRecommended Routes (sorted by priority):")
        print(plan['routes'][['route_id', 'from_station', 'to_station', 
                                'bikes_to_move', 'distance_km', 'priority']].head(10))

# ========================================================================
# SAVE PLAN
# ========================================================================

optimizer.save_plan(plan)

print("\n" + "="*70)
print("REBALANCING PLAN COMPLETE!")
print("="*70)
print("\nNext steps:")
print("1. Review routes in the CSV files")
print("2. Dispatch trucks according to priority")
print("3. Update inventory as bikes are moved")
print("4. Monitor actual vs predicted demand")
print("="*70)